# Susceptibility Distortion Correction for Diffusion Images

Tyler Spears, Tom Fletcher

In [ ]:
!cd "${HOME}/Projects/mr-inr/notebooks/epinr/dmri/"

In [ ]:
# Imports
%autoreload 2
# %matplotlib ipympl
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# By default, allow memory allocation to expand to avoid OOM errors
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = os.environ.get(
    "PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True"
)
# Force torch.load to use 'weights_only=False' by default, unless specified.
# <https://pytorch.org/docs/main/notes/serialization.html#environment-variables>
os.environ["TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD"] = os.environ.get(
    "TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD", "1"
)
# <https://github.com/pytorch/pytorch/issues/121197> and
# <https://github.com/NVIDIA/NeMo-Curator/pull/34>
os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = os.environ.get(
    "TORCHINDUCTOR_COMPILE_THREADS", "1"
)
ENABLE_CUDNN_BENCHMARK = bool(int(os.environ.get("ENABLE_CUDNN_BENCHMARK", "1")))

import math
import random
import time

# from dataclasses import dataclass
import collections
import dataclasses
import tempfile
import copy
from functools import partial
from pathlib import Path
from typing import Any, Optional, Literal, Callable

import torch.multiprocessing as multiprocessing  # noqa

import einops
import numpy as np
import pandas as pd
from box import Box
import yaml

# visualization libraries
# %matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import monai
import monai.data
import monai.transforms
import monai.losses
import kornia

# Computation & ML libraries.
import nibabel as nib
import torch
import torch.func

# The flag below controls whether to allow TF32 on matmul. Defaults to False
# in PyTorch 1.12 and later. See
# <https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices>
# for details.
##
# Set this to True to force full precision matmuls, as registration can be sensitive to
# precision.
torch.backends.cuda.matmul.allow_tf32 = False
torch.set_float32_matmul_precision("highest")
# The flag below controls whether to allow TF32 on cuDNN. Defaults to True.
torch.backends.cudnn.allow_tf32 = False
##
# torch.backends.cuda.matmul.allow_tf32 = True
# torch.set_float32_matmul_precision("medium")
# # The flag below controls whether to allow TF32 on cuDNN. Defaults to True.
# torch.backends.cudnn.allow_tf32 = True
# ##
# # Activate cudnn benchmarking to optimize convolution algorithm speed.
# if torch.backends.cudnn.enabled and ENABLE_CUDNN_BENCHMARK:
#     torch.backends.cudnn.benchmark = True

import mrinr

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from data_utils import (
    PE_DIR_ALIASES,
    DWISubjectData,
    scale_vol,
    blur_mask,
    load_dwi_subject_data,
)

## Function & Model Definitions

In [ ]:
class WeightedNMIParzenLoss(torch.nn.modules.loss._Loss):
    def __init__(
        self,
        num_bins: int = 32,
        sigma_ratio: float = 0.5,
        reduction: str = "mean",
        eps: float = 1e-7,
        norm_mi: bool = True,
        norm_images: bool = True,
    ):
        super().__init__(reduction=reduction)
        if num_bins <= 0:
            raise ValueError("num_bins must > 0, got {num_bins}")
        self.spatial_dims = 3
        self.num_bins = int(num_bins)
        # shape (num_bins)
        bin_centers = torch.linspace(0.0, 1.0, self.num_bins)
        self.register_buffer("bin_centers", bin_centers)
        self.bin_centers: torch.Tensor

        sigma = torch.mean(self.bin_centers[1:] - self.bin_centers[:-1]) * sigma_ratio
        self.register_buffer("sigma", sigma)
        self.sigma: torch.Tensor
        self.eps = eps
        self.norm_mi = norm_mi
        self.norm_images = norm_images

    @staticmethod
    def spatial_normalize(x: torch.Tensor, eps: float) -> torch.Tensor:
        """Min-max normalize x to [0, 1] along spatial dimensions."""
        x_min = einops.reduce(x, "b c x y z -> b c 1 1 1", "min")
        x_max = einops.reduce(x, "b c x y z -> b c 1 1 1", "max")
        x_normalized = (x - x_min) / (x_max - x_min + eps)
        return x_normalized

    def parzen_windowing_gaussian(self, x: torch.Tensor) -> torch.Tensor:
        """Parzen Gaussian weighting function to approximate histogram differentiably.

        Parameters
        ----------
        x : torch.Tensor
            Input tensor of shape B x C x X x Y x Z.

            *NOTE* Input volume must have intensities normalized to [0, 1]. Intensities
            will be clamped to [0, 1] internally.
        Returns
        -------
        torch.Tensor
            Discrete probability distributions for each voxel, shape
            (B*C) x (X*Y*Z) x num_bins. Distributions are not averaged over spatial
            dimensions.
        """
        y = torch.clamp(x, 0.0, 1.0)
        # Move independent dims to the front, and merge spatial dims into a sampling
        # dimension, plus a singleton dimension for broadcasting hist. bins.
        y = einops.rearrange(y, "b c ... -> (b c) (...) 1")
        w_parzen = (1 / (self.sigma * math.sqrt(2 * math.pi))) * torch.exp(
            -0.5 * ((y - self.bin_centers[None, None, :]) / self.sigma) ** 2
        )
        # Normalize over bins.
        p = w_parzen / torch.maximum(
            torch.sum(w_parzen, dim=-1, keepdim=True),
            w_parzen.new_tensor([self.eps]),
        )
        # Wait to average over sampling dimensions until estimating the joint histogram.
        return p

    def weighted_nmi(
        self,
        pred: mrinr.typing.ScalarVolume,
        target: mrinr.typing.ScalarVolume,
        weight_mask: Optional[mrinr.typing.ScalarVolume] = None,
        norm_mi: bool = True,
        normalize_images: bool = True,
    ) -> torch.Tensor:
        """Args:
            pred: the shape should be B[NDHW].
            target: the shape should be same as the pred shape.
            weight_mask: the shape should be B[1DHW] or B[NDHW], optional.
        Raises:
            ValueError: When ``self.reduction`` is not one of ["mean", "sum", "none"].
        """
        if target.shape != pred.shape:
            raise ValueError(
                f"ground truth has differing shape ({target.shape}) from pred ({pred.shape})"
            )

        if normalize_images:
            # Normalize pred and target to [0, 1] along spatial dims, keeping batch and
            # channel dims independent.
            x = self.spatial_normalize(pred, eps=self.eps)
            y = self.spatial_normalize(target, eps=self.eps)
        else:
            x = pred
            y = target

        # Parzen windowing, without averaging over samples. Outputs are
        # shape (B*C)(D*H*W)(num_bins).
        p_pred = self.parzen_windowing_gaussian(x)
        p_target = self.parzen_windowing_gaussian(y)

        # Estimate joint histogram P_pred,target weighted by the weight mask if
        # provided.
        if weight_mask is not None:
            w = einops.rearrange(
                weight_mask.expand_as(pred), "b c x y z -> (b c) (x y z) 1 1"
            )
            # Normalize the weight mask to be in the range [0, 1].
            w = w / torch.maximum(
                w.max(dim=1, keepdim=True).values, w.new_tensor([self.eps])
            )
        else:
            w = 1.0
        p_joint = (
            w * einops.einsum(p_pred, p_target, "bc xyz i, bc xyz j -> bc xyz i j")
        ).sum(1)
        # Normalize joint histogram.
        p_joint = p_joint / torch.maximum(
            p_joint.sum(dim=(-2, -1), keepdim=True),
            p_joint.new_tensor([self.eps]),
        )
        # Estimate marginal histograms.
        p_pred_marginal = p_joint.sum(dim=-1)
        p_target_marginal = p_joint.sum(dim=-2)

        # Compute entropies.
        H_pred = -torch.sum(
            p_pred_marginal
            * torch.log(
                torch.maximum(p_pred_marginal, p_pred_marginal.new_tensor([self.eps]))
            ),
            dim=-1,
        )
        H_target = -torch.sum(
            p_target_marginal
            * torch.log(
                torch.maximum(
                    p_target_marginal, p_target_marginal.new_tensor([self.eps])
                )
            ),
            dim=-1,
        )
        H_joint = -torch.sum(
            p_joint * torch.log(torch.maximum(p_joint, p_joint.new_tensor([self.eps]))),
            dim=(-2, -1),
        )

        # NMI or plain MI.
        if norm_mi:
            mi = (H_pred + H_target) / torch.maximum(
                H_joint, H_joint.new_tensor([self.eps])
            )
        else:
            mi = H_pred + H_target - H_joint

        if self.reduction == "sum":
            # sum over the batch and channel ndims
            r = torch.sum(mi)
        elif self.reduction == "none":
            # No reduction of independent dims.
            r = einops.rearrange(mi, "(b c) -> b c", b=pred.shape[0], c=pred.shape[1])
        elif self.reduction == "mean":
            # average over the batch and channel ndims
            r = torch.mean(mi)
        else:
            raise ValueError(
                f"Unsupported reduction: {self.reduction}, "
                'available options are ["mean", "sum", "none"].'
            )
        return r

    def forward(
        self,
        pred: mrinr.typing.ScalarVolume,
        target: mrinr.typing.ScalarVolume,
        weight_mask: Optional[mrinr.typing.ScalarVolume] = None,
    ) -> torch.Tensor:
        # Loss is negative NMI.
        return -self.weighted_nmi(
            pred,
            target,
            weight_mask=weight_mask,
            norm_mi=self.norm_mi,
            normalize_images=self.norm_images,
        )


class WeightedNCCLoss(torch.nn.modules.loss._Loss):
    def __init__(self, eps: float = 1e-8):
        """Compute weighted normalized cross-correlation (NCC) between two 3D images.
        Args:
            img1, img2: tensors of shape (H, W, D)
            weight: optional tensor of same shape (H, W, D), spatial weights in [0, 1].
                    If None, uniform weights are used.
            eps: small constant for numerical stability.

        Returns:
            scalar tensor: weighted NCC value
        """
        super().__init__(reduction="mean")
        self.eps = eps

    def weighted_ncc(
        self,
        pred: mrinr.typing.ScalarVolume,
        target: mrinr.typing.ScalarVolume,
        weight_mask: Optional[mrinr.typing.ScalarVolume] = None,
    ):
        # assert img1.shape == img2.shape, "Input images must have the same shape"

        # assert x1.shape == x2.shape, "Inputs are not of similar shape"
        # cc = ((x1 - x1.mean()) * (x2 - x2.mean())).mean()
        # stablestd = self._StableStd.apply
        # std = stablestd(x1) * stablestd(x2)
        # ncc = cc / (std + e)
        # return ncc

        x = einops.rearrange(pred, "b c x y z -> b c (x y z)")
        y = einops.rearrange(target, "b c x y z -> b c (x y z)")
        if weight_mask is not None:
            w = einops.rearrange(
                weight_mask.expand_as(pred), "b c x y z -> b c (x y z)"
            )
        else:
            w = torch.ones_like(x)

        if weight is None:
            w = torch.ones_like(i1)
        else:
            w = weight.flatten()
            w = w / (w.sum() + self.eps)  # normalize weights

        # Weighted means
        mu1 = torch.sum(w * i1)
        mu2 = torch.sum(w * i2)

        # Weighted covariance and variances
        v1 = i1 - mu1
        v2 = i2 - mu2

        cov12 = torch.sum(w * v1 * v2)
        var1 = torch.sum(w * v1 * v1)
        var2 = torch.sum(w * v2 * v2)

        # Normalized cross-correlation
        ncc = cov12 / (torch.sqrt(var1 * var2) + self.eps)
        return ncc

    def forward(
        self,
        pred: mrinr.typing.ScalarVolume,
        target: mrinr.typing.ScalarVolume,
        weight_mask: Optional[mrinr.typing.ScalarVolume] = None,
    ) -> torch.Tensor:
        # Loss is negative NCC.
        return -self.weighted_ncc(pred, target, weight_mask=weight_mask)


class CoordSampledWeightedMINDLoss(torch.nn.modules.loss._Loss):
    def __init__(
        self,
        patch_size: int = 3,
        patch_comparison_distance: int = 2,
        reduction: str = "mean",
        eps: float = 1e-7,
        **patch_sampling_kwargs,
    ):
        super().__init__(reduction=reduction)
        self.spatial_dims = 3
        self.eps = eps
        self._patch_sampling_kwargs = patch_sampling_kwargs
        if not self._patch_sampling_kwargs:
            self._patch_sampling_kwargs = dict(
                mode_or_interpolation="linear",
                padding_mode_or_bound="zeros",
                interp_lib="torch",
            )

    @staticmethod
    def spatial_normalize(x: torch.Tensor, eps: float) -> torch.Tensor:
        """Min-max normalize x to [0, 1] along spatial dimensions."""
        x_min = einops.reduce(x, "b c x y z -> b c 1 1 1", "min")
        x_max = einops.reduce(x, "b c x y z -> b c 1 1 1", "max")
        x_normalized = (x - x_min) / (x_max - x_min + eps)
        return x_normalized

    def forward(
        self,
        moving_coords: mrinr.typing.CoordGrid3D,
        fixed_coords: mrinr.typing.CoordGrid3D,
        moving_volume: mrinr.typing.SingleScalarVolume,
        fixed_volume: mrinr.typing.SingleScalarVolume,
        moving_affine: mrinr.typing.SingleHomogeneousAffine3D,
        fixed_affine: mrinr.typing.SingleHomogeneousAffine3D,
        weight_mask: Optional[mrinr.typing.ScalarVolume] = None,
    ) -> torch.Tensor:
        if moving_volume.ndim == 5:
            moving_volume = moving_volume.squeeze(0)
        if fixed_volume.ndim == 5:
            fixed_volume = fixed_volume.squeeze(0)
        if moving_affine.ndim == 3:
            moving_affine = moving_affine.squeeze(0)
        if fixed_affine.ndim == 3:
            fixed_affine = fixed_affine.squeeze(0)

In [ ]:
# Loss functions
class NCC(torch.nn.modules.loss._Loss):
    # Normalized Cross Correlation
    # Taken from <https://github.com/MIAGroupUT/IDIR/blob/main/objectives/ncc.py>,
    # which itself was taken from <https://github.com/BDdeVos/TorchIR>
    class _StableStd(torch.autograd.Function):
        @staticmethod
        def forward(ctx, tensor):
            assert tensor.numel() > 1
            ctx.tensor = tensor.detach()
            res = torch.std(tensor).detach()
            ctx.result = res.detach()
            return res

        @staticmethod
        def backward(ctx, grad_output):
            tensor = ctx.tensor.detach()
            result = ctx.result.detach()
            e = 1e-6
            assert tensor.numel() > 1
            return (
                (2.0 / (tensor.numel() - 1.0))
                * (grad_output.detach() / (result.detach() * 2 + e))
                * (tensor.detach() - tensor.mean().detach())
            )

    def __init__(self, use_mask: bool = False):
        super().__init__()
        self.forward = self.metric

    def ncc(self, x1, x2, e=1e-10):
        assert x1.shape == x2.shape, "Inputs are not of similar shape"
        cc = ((x1 - x1.mean()) * (x2 - x2.mean())).mean()
        stablestd = self._StableStd.apply
        std = stablestd(x1) * stablestd(x2)
        ncc = cc / (std + e)
        return ncc

    def metric(self, fixed: torch.Tensor, warped: torch.Tensor) -> torch.Tensor:
        return -self.ncc(fixed, warped)


class DoGLaplacian(torch.nn.Module):
    """Difference of Gaussians Laplacian filter."""

    def __init__(
        self,
        sigma_low: float | tuple,
        truncate: float | tuple,
        border_type: str = "replicate",
        normalized: bool = False,
    ) -> None:
        super().__init__()

        self.border_type = border_type
        self.normalized = normalized

        if isinstance(sigma_low, (float, int)):
            sigma_low = [sigma_low, sigma_low, sigma_low]
        sigma_low = np.asarray(sigma_low, dtype=np.float32)
        # Scale by 1.6 to get the high sigma that approximates the Laplacian.
        sigma_high = sigma_low * 1.6
        if isinstance(truncate, (float, int)):
            truncate = [truncate, truncate, truncate]
        truncate = np.asarray(truncate, dtype=np.float32)
        kernel_size_low = 2 * np.ceil(truncate * sigma_low).astype(int) + 1
        kernel_size_high = 2 * np.ceil(truncate * sigma_high).astype(int) + 1

        self.sigma_low = tuple(sigma_low.tolist())
        self.sigma_high = tuple(sigma_high.tolist())
        self.kernel_size_low = tuple(kernel_size_low.tolist())
        self.kernel_size_high = tuple(kernel_size_high.tolist())

        gaussian_kernel_low = kornia.filters.get_gaussian_kernel3d(
            kernel_size=kernel_size_low, sigma=self.sigma_low, dtype=torch.float32
        )
        self.register_buffer("gaussian_kernel_low", gaussian_kernel_low)
        self.gaussian_kernel_low: torch.Tensor
        gaussian_kernel_high = kornia.filters.get_gaussian_kernel3d(
            kernel_size=kernel_size_high, sigma=self.sigma_high, dtype=torch.float32
        )
        self.register_buffer("gaussian_kernel_high", gaussian_kernel_high)
        self.gaussian_kernel_high: torch.Tensor

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Apply DoG Laplacian filter to input tensor.

        Args:
            x: Input tensor of shape (B, C, X, Y, Z).

        Returns:
            Tensor of same shape as input, after applying DoG Laplacian filter.
        """
        low_pass = kornia.filters.filter3d(
            x,
            self.gaussian_kernel_low,
            border_type=self.border_type,
            normalized=self.normalized,
        )
        high_pass = kornia.filters.filter3d(
            x,
            self.gaussian_kernel_high,
            border_type=self.border_type,
            normalized=self.normalized,
        )
        dog_laplacian = low_pass - high_pass
        return dog_laplacian


def displace_mag_regularization(
    displacement_field: mrinr.typing.CoordGrid3D,
    norm_ord: int = 2,
) -> torch.Tensor:
    return torch.linalg.norm(displacement_field, ord=norm_ord, dim=-1).mean()


def jacobian_matrix_batched(
    pred_displacement_net: torch.nn.Module,
    x_coords: mrinr.typing.ScalarVolume,
    grid_fov: torch.Tensor,
    grid_min_coord: torch.Tensor,
    jac_of_warp_type: str = "deformation",
    compile_jacobian: bool = False,
):
    jac_of_warp_type = jac_of_warp_type.lower().strip()
    deformation_grad = "deform" in jac_of_warp_type
    displacement_grad = "displac" in jac_of_warp_type
    assert (
        deformation_grad ^ displacement_grad
    ), "Either deformation_grad or displacement_grad must be True."
    is_training = pred_displacement_net.training
    # Set network to eval mode to disable batchnorms, but keep gradients enabled.
    pred_displacement_net.eval()

    # Reshape everything to be of shape B x 3, to make the vmap easier.
    orig_x_shape = tuple(x_coords.shape)
    x_coords = einops.rearrange(
        x_coords,
        "b x y z coord -> (b x y z) coord",
        coord=3,
    )
    grid_fov = einops.rearrange(
        grid_fov,
        "b x y z coord -> (b x y z) coord",
        coord=3,
    )
    grid_min_coord = einops.rearrange(
        grid_min_coord,
        "b x y z coord -> (b x y z) coord",
        coord=3,
    )
    # Create function for the Jacobian of the *deformation* field using
    # reverse-mode auto-diff.
    if deformation_grad:
        jac_fn = torch.func.jacrev(
            lambda x, f, m: pred_displacement_net(x, f, m) + x, argnums=0
        )
    else:
        jac_fn = torch.func.jacrev(
            lambda x, f, m: pred_displacement_net(x, f, m), argnums=0
        )
    # Vectorize the Jacobian function over the batch size.
    # Ensure all Tensors have the same batch size.
    grid_fov = grid_fov.expand_as(x_coords)
    grid_min_coord = grid_min_coord.expand_as(x_coords)
    v_jac_fn = torch.vmap(jac_fn, in_dims=(0, 0, 0), out_dims=0, randomness="error")
    if compile_jacobian:
        v_jac_fn = torch.compile(v_jac_fn)

    # Compute the Jacobian.
    J = v_jac_fn(x_coords, grid_fov, grid_min_coord)
    # Reshape the batch dims to match the input shape.
    J = einops.rearrange(
        J,
        "(b x y z) n_coord m_coord -> b x y z n_coord m_coord",
        b=orig_x_shape[0],
        x=orig_x_shape[1],
        y=orig_x_shape[2],
        z=orig_x_shape[3],
    )

    pred_displacement_net.train(is_training)
    return J


def bending_energy_regularization(
    pred_displacement_net: torch.nn.Module,
    x_coords: mrinr.typing.ScalarVolume,
    grid_fov: torch.Tensor,
    grid_min_coord: torch.Tensor,
    pe_dir: str,
    hessian_of_warp_type: str = "deformation",
    compile_hessian: bool = False,
) -> torch.Tensor:
    hess_of_warp_type = hessian_of_warp_type.lower().strip()
    deformation_grad = "deform" in hess_of_warp_type
    displacement_grad = "displac" in hess_of_warp_type
    assert (
        deformation_grad ^ displacement_grad
    ), "Either deformation_grad or displacement_grad must be True."
    is_training = pred_displacement_net.training
    # Set network to eval mode to disable batchnorms, but keep gradients enabled.
    pred_displacement_net.eval()

    # Reshape everything to be of shape B x 3, to make the vmap easier.
    orig_x_shape = tuple(x_coords.shape)
    x_coords = einops.rearrange(
        x_coords,
        "b x y z coord -> (b x y z) coord",
        coord=3,
    )
    orig_grid_fov = grid_fov
    grid_fov = einops.rearrange(
        orig_grid_fov,
        "b x y z coord -> (b x y z) coord",
        coord=3,
    )
    grid_min_coord = einops.rearrange(
        grid_min_coord,
        "b x y z coord -> (b x y z) coord",
        coord=3,
    )
    # Create function for the Hessian of the *deformation* field using
    # reverse-mode auto-diff.
    if deformation_grad:
        hessian_fn = torch.func.hessian(
            lambda x, f, m: pred_displacement_net(x, f, m) + x, argnums=0
        )
    else:
        hessian_fn = torch.func.hessian(
            lambda x, f, m: pred_displacement_net(x, f, m), argnums=0
        )

    pe_dir = PE_DIR_ALIASES[pe_dir]
    # The Hessian should only be non-zero along the PE direction.
    if pe_dir in {"ap", "pa"}:
        hessian_fn_pe = lambda x, f, m: hessian_fn(x, f, m)[
            ..., 1
        ]  # Y coordinate of the output vector.
    else:
        raise ValueError(f"Unsupported PE direction: {pe_dir}")

    # Vectorize the Hessian function over the batch size.
    # Ensure all Tensors have the same batch size.
    grid_fov = grid_fov.expand_as(x_coords)
    grid_min_coord = grid_min_coord.expand_as(x_coords)
    v_hessian_fn_pe = torch.vmap(
        hessian_fn_pe, in_dims=(0, 0, 0), out_dims=0, randomness="error"
    )
    if compile_hessian:
        v_hessian_fn_pe = torch.compile(v_hessian_fn_pe)
    # Compute the full 3D, batched Hessian.
    H_pe = v_hessian_fn_pe(x_coords, grid_fov, grid_min_coord)
    # Reshape the dims to match the input shape.
    # Shape of Hessian tensor without batch dims:
    # out_dims_m x in_dims_n x in_dims_n
    # which is just a batch of Hessian matrices such that for every output dimension k,
    # there is a Hessian matrix H_k of all input dimension combinations:
    # H_k_ij = d^2 f_k / (d x_i d x_j)
    H_pe = einops.rearrange(
        H_pe,
        "(b x y z) n_coord_1 n_coord_2 -> b x y z n_coord_1 n_coord_2",
        b=orig_x_shape[0],
        x=orig_x_shape[1],
        y=orig_x_shape[2],
        z=orig_x_shape[3],
    )

    # Normalizing domain volume/size.
    # V = orig_grid_fov.reshape(orig_x_shape[0], 3).prod(dim=-1)
    # # Compute bending energy.
    # # Sum over all spatial locations and coordinate directions, normalizing by batch-wise
    # # FOV/volume.
    # l_bending = (
    #     einops.reduce(
    #         H_pe**2,
    #         "b x y z n_coord_1 n_coord_2 -> b",
    #         "sum",
    #         b=orig_x_shape[0],
    #         x=orig_x_shape[1],
    #         y=orig_x_shape[2],
    #         z=orig_x_shape[3],
    #         n_coord_1=3,
    #         n_coord_2=3,
    #     )
    #     / V
    # )
    #
    # Compute bending energy.
    # Sum over all spatial locations and coordinate directions, normalizing by batch-wise
    # FOV/volume.
    l_bending = einops.reduce(
        H_pe**2,
        "b x y z n_coord_1 n_coord_2 -> b x y z",
        "sum",
        b=orig_x_shape[0],
        x=orig_x_shape[1],
        y=orig_x_shape[2],
        z=orig_x_shape[3],
        n_coord_1=3,
        n_coord_2=3,
    ).mean()
    #

    pred_displacement_net.train(is_training)
    return l_bending


def smooth_displace_regularization(J_displacement: torch.Tensor) -> torch.Tensor:
    spatial_dims = 3
    J = J_displacement
    # Last 2 dimensions of J should be the coordinate dimensions.
    assert J.shape[-1] == J.shape[-2] == spatial_dims
    J = J.view(-1, spatial_dims, spatial_dims)
    # Correct implementation?
    l_smooth = torch.linalg.matrix_norm(J, ord="fro") ** 2
    l_smooth = (l_smooth / spatial_dims).mean()
    return l_smooth


def jac_deform_regularization(J_deform: torch.Tensor) -> torch.Tensor:
    spatial_dims = 3
    # Compute the Jacobian determinant for regularization.
    # Last 2 dimensions of J should be the coordinate dimensions.
    J = J_deform
    assert J.shape[-1] == J.shape[-2] == spatial_dims
    J = J.view(-1, spatial_dims, spatial_dims)
    batch_size = J.shape[0]
    l = torch.abs(1 - torch.linalg.det(J)).sum()
    l /= batch_size
    return l


def jac_displace_fro_regularization(J_displace: torch.Tensor) -> torch.Tensor:
    spatial_dims = 3
    # Compute the Jacobian determinant for regularization.
    # Last 2 dimensions of J should be the coordinate dimensions.
    J = J_displace
    assert J.shape[-1] == J.shape[-2] == spatial_dims
    J = J.view(-1, spatial_dims, spatial_dims)
    l = torch.linalg.matrix_norm(J, ord="fro", dim=(-2, -1)) ** 2
    l = l.mean()
    return l

In [ ]:
class Siren(torch.nn.Module):
    """SIREN implicit neural representation.

    From:
    V. Sitzmann, J. N. P. Martel, A. W. Bergman, D. B. Lindell, and G. Wetzstein,
        “Implicit Neural Representations with Periodic Activation Functions.”
        arXiv, Jun. 17, 2020. doi: 10.48550/arXiv.2006.09661.

    Implementation adapated from <https://github.com/lucidrains/siren-pytorch>
    """

    def __init__(
        self,
        in_features,
        hidden_features,
        out_features,
        num_layers,
        omega=1.0,
        omega_0=30.0,
        c: float = 6.0,
        bias=True,
    ):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_features = hidden_features
        self.in_features = in_features
        self.out_features = out_features
        self.c = c

        layers = list()
        for idx in range(num_layers):
            is_first = idx == 0
            omega_i = omega_0 if is_first else omega
            in_features_i = self.in_features if is_first else self.hidden_features

            layer = SirenBlock(
                omega=omega_i,
                is_first_layer=is_first,
                c=self.c,
                use_activation=True,
                in_features=in_features_i,
                out_features=self.hidden_features,
                bias=bias,
            )

            layers.append(layer)

        # Final layer without activation or bias. As a displacement field, there should
        # be no global translation component, so the bias is disabled.
        final_layer = SirenBlock(
            omega=omega,
            c=self.c,
            is_first_layer=False,
            use_activation=False,
            in_features=self.hidden_features,
            out_features=self.out_features,
            bias=False,
        )
        layers.append(final_layer)

        self.linear_layers = torch.nn.Sequential(*layers)

    def forward(self, x, return_intermediates: bool = False):
        if not return_intermediates:
            y = self.linear_layers(x)
        else:
            y_i = x
            intermediates = list()
            for lin in self.linear_layers:
                y_i = lin(y_i)
                intermediates.append(y_i)
            y = intermediates[-1], tuple(intermediates[:-1])
        return y


class SirenBlock(torch.nn.Module):
    def __init__(
        self,
        omega: float,
        is_first_layer: bool,
        c: float = 6,
        use_activation: bool = True,
        activate_fn_kwargs: dict = dict(),
        **linear_kwargs,
    ):
        """SIREN implicit neural representation linear and activation block.

        From:
        V. Sitzmann, J. N. P. Martel, A. W. Bergman, D. B. Lindell, and G. Wetzstein,
            “Implicit Neural Representations with Periodic Activation Functions.”
            arXiv, Jun. 17, 2020. doi: 10.48550/arXiv.2006.09661.

        Implementation adapated from <https://github.com/lucidrains/siren-pytorch>
        and <https://github.com/vsitzmann/siren/blob/master/explore_siren.ipynb>.

        Parameters
        ----------
        omega : float
            Sine scale parameter; determines sine weight and linear weight init.

            In the original paper, omega = 30 was chosen for many tests.
        is_first_layer : bool
            Indicator for being the first layer in the INR MLP.
        c : float, optional
            Constant to drive initial weight distribution, by default 6.

            This should probably not be changed, but it probably shouldn't be hardcoded.
        """

        super().__init__()

        self.linear = torch.nn.Linear(**linear_kwargs)
        if use_activation:
            self.activate_fn = mrinr.nn.Sine(omega, **activate_fn_kwargs)
        else:
            self.activate_fn = None

        self._init_siren_linear_(
            self.linear, omega=omega, is_first_layer=is_first_layer, c=c
        )

    def forward(self, x):
        y = self.linear(x)
        if self.activate_fn is not None:
            y = self.activate_fn(y)
        return y

    @staticmethod
    def _init_siren_linear_(
        l: torch.nn.Linear,
        omega: float,
        is_first_layer: bool,
        c: float,
    ):
        n = l.in_features
        if is_first_layer:
            omega_bound = 1 / n
        else:
            omega_bound = math.sqrt(c / n) / omega
        torch.nn.init.uniform_(l.weight, -omega_bound, omega_bound)
        if l.bias is not None:
            torch.nn.init.uniform_(l.bias, -omega_bound, omega_bound)


class ImageReconINR(torch.nn.Module):
    def __init__(
        self,
        hidden_features: int,
        num_layers: int,
        out_features: int,
        omega=1.0,
        omega_0=30.0,
        c: float = 6.0,
        bias=True,
        pos_enc: Optional[Literal["positional"] | Literal["gaussian"]] = None,
        sigma: Optional[float] = None,
        m_num_freqs: Optional[int] = None,
    ):
        self._init_kwargs = mrinr.nn.get_module_init_kwargs(
            locals(), extra_kwargs_dict=dict()
        )
        super().__init__()
        self.spatial_dims = 3
        self.in_features = self.spatial_dims
        self.out_features = out_features

        self.pos_enc_sigma = sigma
        self.m_num_freqs = m_num_freqs
        if pos_enc == "positional":
            self.pos_enc = mrinr.nn.PositionalCoordEncoding(
                spatial_dims=self.spatial_dims,
                sigma_scale=self.pos_enc_sigma,
                m_num_freqs=self.m_num_freqs,
            )
            siren_in_features = self.pos_enc.out_features
        elif pos_enc == "gaussian":
            self.pos_enc = mrinr.nn.GaussianCoordEncodingPreSampled(
                spatial_dims=self.spatial_dims,
                sigma=self.pos_enc_sigma,
                m_num_freqs=self.m_num_freqs,
            )
            siren_in_features = self.pos_enc.out_features
        else:
            self.pos_enc = None
            siren_in_features = self.spatial_dims

        self.siren = Siren(
            in_features=siren_in_features,
            hidden_features=hidden_features,
            out_features=self.out_features,
            num_layers=num_layers,
            omega=omega,
            omega_0=omega_0,
            c=c,
            bias=bias,
        )

    def get_extra_state(self) -> Any:
        return {"init_kwargs": self._init_kwargs}

    def set_extra_state(self, state):
        return

    def _get_reshape_fns(
        self,
        x: mrinr.typing.CoordGrid3D | torch.Tensor,
        grid_fov: torch.Tensor,
        grid_min_coord: torch.Tensor,
    ) -> tuple[Callable, Callable]:
        """Get functions to reshape coordinates to/from grids.

        Helps with handling inputs that are either full coordinate grids or just
        batches of coordinates, the latter of which is useful for Jacobian/Hessian computations."""
        assert (
            x.ndim == grid_fov.ndim == grid_min_coord.ndim
        ), f"Input shapes mismatch: {x.shape =}, {grid_fov.shape =}, {grid_min_coord.shape =}"
        x_orig_shape = tuple(x.shape)
        if x.ndim <= 2 and x.shape[-1] == self.spatial_dims:
            if x.ndim == 1:
                coords_to_grid_reshape = lambda a: einops.repeat(
                    a,
                    "coord -> b x y z coord",
                    b=1,
                    x=1,
                    y=1,
                    z=1,
                    coord=self.spatial_dims,
                )
                grid_to_coords_reshape = lambda a: einops.rearrange(
                    a,
                    "b x y z coord -> (b x y z) coord",
                    b=1,
                    x=1,
                    y=1,
                    z=1,
                    coord=self.spatial_dims,
                ).squeeze(0)
            else:
                coords_to_grid_reshape = lambda a: einops.rearrange(
                    a,
                    "(b x y z) coord -> b x y z coord",
                    x=1,
                    y=1,
                    z=1,
                    coord=self.spatial_dims,
                )
                grid_to_coords_reshape = lambda a: einops.rearrange(
                    a,
                    "b x y z coord -> (b x y z) coord",
                    b=x_orig_shape[0],
                    coord=self.spatial_dims,
                )
        else:
            # Identity functions.
            coords_to_grid_reshape = lambda a: a
            grid_to_coords_reshape = lambda a: a
        return coords_to_grid_reshape, grid_to_coords_reshape

    def forward(
        self,
        x: mrinr.typing.CoordGrid3D | torch.Tensor,
        grid_fov: torch.Tensor,
        grid_min_coord: torch.Tensor,
        return_intermediates: bool = False,
    ) -> (
        mrinr.typing.ScalarVolume
        | tuple[mrinr.typing.ScalarVolume, tuple[mrinr.typing.ScalarVolume]]
    ):
        # print(f"{x.shape =}, {grid_fov.shape =}, {grid_min_coord.shape =}")
        # Inputs should be 3D coordinate grids, but if they are just a batch of 3D
        # coordinates, then reshape to a grid.
        coords_to_grid_reshape, _ = self._get_reshape_fns(x, grid_fov, grid_min_coord)
        x = coords_to_grid_reshape(x)
        grid_fov = coords_to_grid_reshape(grid_fov)
        grid_min_coord = coords_to_grid_reshape(grid_min_coord)

        # Scale the input coordinates to the range [0, 1] based on the entire volume's
        # field of view and lowest coordinate.
        # norm_x = (x - grid_min_coord) / grid_fov
        # Scale the input coordinates to the range [-1, 1] based on the entire volume's
        # field of view and lowest coordinate.
        norm_x = (((x - grid_min_coord) / grid_fov) * 2.0) - 1.0

        # Positional encoding of input coordinates.
        if self.pos_enc is not None:
            x_enc = self.pos_enc(norm_x)
        else:
            x_enc = mrinr.nn.coords_as_channels(norm_x, has_batch_dim=True)
        x_enc = einops.rearrange(x_enc, "b c x y z -> (b x y z) c")

        if return_intermediates:
            y, inters = self.siren(x_enc, return_intermediates=True)
            y_intermediates = list()
            for a in inters:
                a = einops.rearrange(
                    a,
                    "(b x y z) c -> b c x y z",
                    x=x.shape[1],
                    y=x.shape[2],
                    z=x.shape[3],
                )
                y_intermediates.append(a)
            y_intermediates = tuple(y_intermediates)
        else:
            y = self.siren(x_enc)
            y_intermediates = None
        # Reshape back into a volume, and return.
        y = einops.rearrange(
            y,
            "(b x y z) c -> b c x y z",
            x=x.shape[1],
            y=x.shape[2],
            z=x.shape[3],
        )

        if return_intermediates:
            r = y, y_intermediates
        else:
            r = y
        return r

In [ ]:
# EPINR SIREN model with optional positional encoding and rigid affine layer.


class EPINR(torch.nn.Module):
    def __init__(
        self,
        hidden_features: int,
        num_layers: int,
        pe_dir: str,
        omega=1.0,
        omega_0=30.0,
        c: float = 6.0,
        non_rigid_affine: bool = False,
        non_rigid_affine_nonzero_thresh: Optional[float] = None,
        refine_fixed_rigid_affine: bool = False,
        bias=True,
        pos_enc: Optional[Literal["positional"] | Literal["gaussian"]] = None,
        sigma: Optional[float] = None,
        m_num_freqs: Optional[int] = None,
    ):
        self._init_kwargs = mrinr.nn.get_module_init_kwargs(
            locals(), extra_kwargs_dict=dict()
        )
        super().__init__()
        self.spatial_dims = 3
        self.in_features = self.spatial_dims
        self.pe_dir = pe_dir
        if self.pe_dir not in {"ap", "pa"}:
            raise NotImplementedError(f"Unsupported pe_dir: {self.pe_dir}")

        if non_rigid_affine:
            self.nonrigid_affine_layer = self._NonRigidAffineLayer(
                pe_dir=self.pe_dir,
                param_softshrink_lambda=non_rigid_affine_nonzero_thresh,
            )
        else:
            self.nonrigid_affine_layer = None

        self.out_features = 1
        self.pos_enc_sigma = sigma
        self.m_num_freqs = m_num_freqs
        if pos_enc == "positional":
            self.pos_enc = mrinr.nn.PositionalCoordEncoding(
                spatial_dims=self.spatial_dims,
                sigma_scale=self.pos_enc_sigma,
                m_num_freqs=self.m_num_freqs,
            )
            siren_in_features = self.pos_enc.out_features
        elif pos_enc == "gaussian":
            self.pos_enc = mrinr.nn.GaussianCoordEncodingPreSampled(
                spatial_dims=self.spatial_dims,
                sigma=self.pos_enc_sigma,
                m_num_freqs=self.m_num_freqs,
            )
            siren_in_features = self.pos_enc.out_features
        else:
            self.pos_enc = None
            siren_in_features = self.spatial_dims

        self.siren = Siren(
            in_features=siren_in_features,
            hidden_features=hidden_features,
            out_features=self.out_features,
            num_layers=num_layers,
            omega=omega,
            omega_0=omega_0,
            c=c,
            bias=bias,
        )
        #!TESTING
        # Initialize the final layer weights to be very small to start as a small
        # displacement field.
        with torch.no_grad():
            final_layer_init_range = 0.0001  # 0.000001
            torch.nn.init.uniform_(
                self.siren.linear_layers[-1].linear.weight,
                -final_layer_init_range,
                final_layer_init_range,
            )
        #!
        if refine_fixed_rigid_affine:
            self.fixed_rigid_affine = self._RigidAffineLayer()
        else:
            self.fixed_rigid_affine = None

    class _RigidAffineLayer(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.spatial_dims = 3
            self.theta = torch.nn.Parameter(torch.zeros(self.spatial_dims))
            self.translations = torch.nn.Parameter(torch.zeros(self.spatial_dims))

        def get_affine(self, homogeneous: bool = False):
            R_x = torch.eye(self.spatial_dims).to(self.theta)
            # [1, 0, 0],
            # [0, torch.cos(self.theta[0]), -torch.sin(self.theta[0])],
            # [0, torch.sin(self.theta[0]), torch.cos(self.theta[0])],
            R_x[1, 1] *= torch.cos(self.theta[0])
            R_x[1, 2] += -torch.sin(self.theta[0])
            R_x[2, 1] += torch.sin(self.theta[0])
            R_x[2, 2] *= torch.cos(self.theta[0])

            R_y = torch.eye(self.spatial_dims).to(self.theta)
            # [torch.cos(self.theta[1]), 0, torch.sin(self.theta[1])],
            # [0, 1, 0],
            # [-torch.sin(self.theta[1]), 0, torch.cos(self.theta[1])]
            R_y[0, 0] *= torch.cos(self.theta[1])
            R_y[0, 2] += torch.sin(self.theta[1])
            R_y[2, 0] += -torch.sin(self.theta[1])
            R_y[2, 2] *= torch.cos(self.theta[1])

            R_z = torch.eye(self.spatial_dims).to(self.theta)
            # [torch.cos(self.theta[2]), -torch.sin(self.theta[2]), 0],
            # [torch.sin(self.theta[2]), torch.cos(self.theta[2]), 0],
            # [0, 0, 1],
            R_z[0, 0] *= torch.cos(self.theta[2])
            R_z[0, 1] += -torch.sin(self.theta[2])
            R_z[1, 0] += torch.sin(self.theta[2])
            R_z[1, 1] *= torch.cos(self.theta[2])

            R = R_x @ R_y @ R_z
            if homogeneous:
                A = torch.eye(
                    self.spatial_dims + 1,
                    device=self.translations.device,
                    dtype=self.translations.dtype,
                )
                A[: self.spatial_dims, : self.spatial_dims] = R
                A[:-1, -1] = self.translations
                r = A
            else:
                r = R, self.translations
            return r

        def forward(self, x):
            R, t = self.get_affine(homogeneous=False)
            y = einops.einsum(x, R, "... i, i j -> ... j") + t
            # linear() requires a transposed matrix.
            # y = torch.nn.functional.linear(x, weight=R.T, bias=self.translations)
            return y

    class _NonRigidAffineLayer(torch.nn.Module):
        def __init__(
            self,
            pe_dir: str,
            param_softshrink_lambda: Optional[float] = None,
        ):
            super().__init__()
            self.spatial_dims = 3
            self.pe_dir = pe_dir
            if self.pe_dir not in {"ap", "pa"}:
                raise NotImplementedError(f"Unsupported pe_dir: {self.pe_dir}")
            self.scales = torch.nn.Parameter(torch.ones(self.spatial_dims))
            self.shears = torch.nn.Parameter(torch.zeros(self.spatial_dims))
            if (param_softshrink_lambda is None) or (param_softshrink_lambda == 0.0):
                self.param_nonlinear = None
            else:
                self.param_nonlinear = torch.nn.Softshrink(
                    lambd=param_softshrink_lambda
                )

        def get_affine(self, homogeneous: bool = False):
            # Apply nonlinear to parameters so they remain at identity when only small
            # updates are made, ensuring that large gradients are needed to "activate"
            # the affine transformation.
            if self.param_nonlinear is not None:
                scales = self.param_nonlinear(self.scales - 1.0) + 1.0
                shears = self.param_nonlinear(self.shears)
            else:
                scales = self.scales
                shears = self.shears
            # Scaling matrix.
            S = torch.diag(scales)
            # Shear matrix.
            P = torch.eye(self.spatial_dims, dtype=shears.dtype, device=shears.device)
            P[0, 1] = shears[0]  # shear xy
            # shear xz, which is 0 for AP/PA PE directions.
            P[0, 2] = shears[1] * 0.0
            P[1, 2] = shears[2]  # shear yz

            A = S @ P
            if homogeneous:
                A = torch.nn.functional.pad(A, pad=(0, 1, 0, 1), value=0.0)
                A[-1, -1] = 1.0
            return A

    def get_extra_state(self) -> Any:
        return {"init_kwargs": self._init_kwargs}

    def set_extra_state(self, state):
        return

    def fixed2moving_rigid_affine_parameters(self):
        if self.fixed_rigid_affine is not None:
            return self.fixed_rigid_affine.parameters()
        else:
            return list()

    def moving2fixed_nonrigid_affine_parameters(self):
        if self.nonrigid_affine_layer is not None:
            return self.nonrigid_affine_layer.parameters()
        else:
            return list()

    def network_parameters(self):
        p_iter_all = self.parameters()
        fix2mov_rigid_params = list(self.fixed2moving_rigid_affine_parameters())
        mov2fix_nonrigid_params = list(self.moving2fixed_nonrigid_affine_parameters())
        affine_params = set(fix2mov_rigid_params + mov2fix_nonrigid_params)
        net_params = [p for p in p_iter_all if p not in affine_params]
        return net_params

    @property
    def fixed2moving_rigid_affine(
        self,
    ) -> mrinr.typing.SingleHomogeneousAffine3D | None:
        if self.fixed_rigid_affine is not None:
            A = self.fixed_rigid_affine.get_affine(homogeneous=True)
        else:
            A = None
        return A

    def apply_fixed_rigid_affine(
        self, x: mrinr.typing.CoordGrid3D
    ) -> mrinr.typing.CoordGrid3D:
        if self.fixed_rigid_affine is not None:
            y = self.fixed_rigid_affine(x)
        else:
            y = x
        return y

    def _get_reshape_fns(
        self,
        x: mrinr.typing.CoordGrid3D | torch.Tensor,
        grid_fov: torch.Tensor,
        grid_min_coord: torch.Tensor,
    ) -> tuple[Callable, Callable]:
        """Get functions to reshape coordinates to/from grids.

        Helps with handling inputs that are either full coordinate grids or just
        batches of coordinates, the latter of which is useful for Jacobian/Hessian computations."""
        assert (
            x.ndim == grid_fov.ndim == grid_min_coord.ndim
        ), f"Input shapes mismatch: {x.shape =}, {grid_fov.shape =}, {grid_min_coord.shape =}"
        x_orig_shape = tuple(x.shape)
        if x.ndim <= 2 and x.shape[-1] == self.spatial_dims:
            if x.ndim == 1:
                coords_to_grid_reshape = lambda a: einops.repeat(
                    a,
                    "coord -> b x y z coord",
                    b=1,
                    x=1,
                    y=1,
                    z=1,
                    coord=self.spatial_dims,
                )
                grid_to_coords_reshape = lambda a: einops.rearrange(
                    a,
                    "b x y z coord -> (b x y z) coord",
                    b=1,
                    x=1,
                    y=1,
                    z=1,
                    coord=self.spatial_dims,
                ).squeeze(0)
            else:
                coords_to_grid_reshape = lambda a: einops.rearrange(
                    a,
                    "(b x y z) coord -> b x y z coord",
                    x=1,
                    y=1,
                    z=1,
                    coord=self.spatial_dims,
                )
                grid_to_coords_reshape = lambda a: einops.rearrange(
                    a,
                    "b x y z coord -> (b x y z) coord",
                    b=x_orig_shape[0],
                    coord=self.spatial_dims,
                )
        else:
            # Identity functions.
            coords_to_grid_reshape = lambda a: a
            grid_to_coords_reshape = lambda a: a
        return coords_to_grid_reshape, grid_to_coords_reshape

    def forward(
        self,
        x: mrinr.typing.CoordGrid3D | torch.Tensor,
        grid_fov: torch.Tensor,
        grid_min_coord: torch.Tensor,
        _scale_by_fov: bool = True,
        _add_nonrigid: bool = True,
    ) -> mrinr.typing.CoordGrid3D:
        # print(f"{x.shape =}, {grid_fov.shape =}, {grid_min_coord.shape =}")
        # Inputs should be 3D coordinate grids, but if they are just a batch of 3D
        # coordinates, then reshape to a grid.
        coords_to_grid_reshape, grid_to_coords_reshape = self._get_reshape_fns(
            x, grid_fov, grid_min_coord
        )
        x = coords_to_grid_reshape(x)
        grid_fov = coords_to_grid_reshape(grid_fov)
        grid_min_coord = coords_to_grid_reshape(grid_min_coord)

        # Use a learned non-rigid affine transformation to create a secondary displacement
        # field.
        if self.nonrigid_affine_layer is not None:
            # Broadcast batch dim.
            A = self.nonrigid_affine_layer.get_affine(homogeneous=True).unsqueeze(0)
            Ax = mrinr.coords.transform_coords(x, affine_a2b=A, broadcast_batch=True)
            y_nonrigid = Ax - x
        else:
            y_nonrigid = None

        # Scale the input coordinates to the range [-1, 1] based on the entire volume's
        # field of view and lowest coordinate.
        norm_x = (((x - grid_min_coord) / grid_fov) * 2.0) - 1.0

        # Positional encoding of input coordinates.
        if self.pos_enc is not None:
            x_enc = self.pos_enc(norm_x)
        else:
            x_enc = mrinr.nn.coords_as_channels(norm_x, has_batch_dim=True)
        x_enc = einops.rearrange(x_enc, "b c x y z -> (b x y z) c")
        y = self.siren(x_enc)
        # Reshape back into a coordinate grid, and return.
        y = einops.rearrange(
            y,
            "(b x y z) coord -> b x y z coord",
            x=x.shape[1],
            y=x.shape[2],
            z=x.shape[3],
        )

        y = torch.cat([torch.zeros_like(y), y, torch.zeros_like(y)], dim=-1)
        # Unscale the output coordinates to the grid.
        # y = (y * grid_fov) + grid_min_coord
        if _scale_by_fov:
            y = y * grid_fov
        if y_nonrigid is not None and _add_nonrigid:
            y = y + y_nonrigid
        y = grid_to_coords_reshape(y)
        return y

## Data Processing & Loading

In [ ]:
def prep_vols(
    b0,
    b0_mask,
    b0_affine,
    t1w,
    t1w_mask,
    t1w_affine,
    scale_kwargs: dict,
    blur_mask_kwargs: dict,
) -> dict:
    # Scale the volumes to have similar intensity ranges.
    b0 = scale_vol(b0, **scale_kwargs)
    t1w = scale_vol(t1w, **scale_kwargs)
    # Blur masks to focus training loss on brain tissue instead of skull/background.
    b0_spacing = mrinr.coords.spacing(b0_affine)
    b0_mask = blur_mask(b0_mask, spacing=b0_spacing, **blur_mask_kwargs)
    t1w_spacing = mrinr.coords.spacing(t1w_affine)
    t1w_mask = blur_mask(t1w_mask, spacing=t1w_spacing, **blur_mask_kwargs)
    vols = dict(
        b0=b0,
        b0_mask=b0_mask,
        t1w=t1w,
        t1w_mask=t1w_mask,
    )
    return vols

In [ ]:
class RepeatVolPatchDataset(monai.data.IterableDataset):
    def __init__(
        self,
        vol_data: dict,
        n_repeats: int,
        starting_epoch: int = 1,
        **patch_iterd_kwargs,
    ):
        # See <https://docs.monai.io/en/stable/data.html#patchiterd> for PatchIterd
        # kwargs.
        self._data = vol_data
        self.n_repeats = n_repeats
        self._starting_epoch = starting_epoch
        self._epoch = self._starting_epoch
        self._patch_iterd_kwargs = patch_iterd_kwargs
        ex_iter = monai.data.PatchIterd(**self._patch_iterd_kwargs)
        self.n_patches_in_vol = len(list(ex_iter(self._data)))

    def __iter__(self):
        info = torch.utils.data.get_worker_info()
        num_workers = info.num_workers if info is not None else 1
        id = info.id if info is not None else 0

        for i in range(self.n_repeats):
            if i % num_workers == id:
                for j, (item, coords) in enumerate(
                    monai.data.PatchIterd(**self._patch_iterd_kwargs)(self._data)
                ):
                    yield item
        self._epoch += 1


class RandSingleVolCoordSampleDataset(
    monai.data.iterable_dataset.Randomizable, monai.data.IterableDataset
):
    def __init__(
        self,
        coord_grid: mrinr.typing.SingleCoordGrid3D,
        weight_mask: mrinr.typing.SingleScalarVolume,
        n_samples_per_batch: int,
        n_batches_per_epoch: int,
        seed: int = 1,
        p_rand_coord_shift: float = 0.0,
        rand_shift_ranges: Optional[tuple[float, float, float]] = None,
        starting_epoch: int = 1,
    ):
        super().__init__(data=[coord_grid], transform=None)
        self._coords = coord_grid
        self._coords_as_vol = mrinr.nn.coords_as_channels(
            self._coords, has_batch_dim=False
        )

        self._weight_mask = weight_mask
        self._cumul_weight_mask = mrinr.data.cumul_sampling_mask_from_weight_mask(
            weight_mask=weight_mask
        )
        self.n_samples_per_batch = n_samples_per_batch
        self.n_batches_per_epoch = n_batches_per_epoch
        self._starting_epoch = starting_epoch
        self._epoch = self._starting_epoch

        self.seed = seed
        self.p_rand_coord_shift = p_rand_coord_shift
        self.rand_shift_ranges = rand_shift_ranges
        if isinstance(self.rand_shift_ranges, float):
            self.rand_shift_ranges = [self.rand_shift_ranges] * 3
        if self.p_rand_coord_shift > 0.0 and self.rand_shift_ranges is None:
            raise ValueError(
                "If p_rand_coord_shift > 0.0, rand_shift_ranges must be provided."
            )
        if self.rand_shift_ranges is not None:
            self.rand_shift_ranges = tuple(self.rand_shift_ranges)
            if len(self.rand_shift_ranges) != 3:
                raise ValueError("rand_shift_ranges must be a tuple of 3 floats.")
        self.R.seed(self.seed)
        self.random_vars = None
        self.rand_coord_shifts = None

    def randomize(self, size: tuple) -> None:
        self.random_vars = torch.from_numpy(self.R.random_sample(size)).to(
            self._cumul_weight_mask
        )
        if self.p_rand_coord_shift > 0.0:
            do_rand_shift = self.R.random_sample(size) < self.p_rand_coord_shift
            rand_shifts = list()
            for dim in range(3):
                shift_range = self.rand_shift_ranges[dim]
                shifts_dim = self.R.uniform(
                    -shift_range, shift_range, size=size
                ).astype(np.float32)
                rand_shifts.append(shifts_dim)
            rand_shifts = np.stack(rand_shifts, axis=-1)
            rand_shifts = rand_shifts * do_rand_shift[..., None]
            self.rand_coord_shifts = torch.from_numpy(rand_shifts).to(
                self._cumul_weight_mask
            )
        else:
            self.rand_coord_shifts = None

    def get_random_coords_from_rv(
        self, random_vars: torch.Tensor, coord_shifts: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        rv = random_vars.view(-1)
        patch_center_flat_indices = torch.searchsorted(
            self._cumul_weight_mask.view(-1), rv, side="right"
        ).clamp_max_(self._cumul_weight_mask.numel() - 1)
        patch_center_indices = np.unravel_index(
            patch_center_flat_indices.cpu().numpy(),
            shape=tuple(self._cumul_weight_mask.shape),
        )
        patch_center_el_coords = torch.from_numpy(
            np.asarray(patch_center_indices)
        ).T.to(self._cumul_weight_mask.device)
        # Ensure coords are within valid range.
        max_idx = patch_center_el_coords.new_tensor(self._coords_as_vol.shape[1:]) - 1
        patch_center_el_coords = torch.minimum(
            patch_center_el_coords, max_idx.reshape(1, 3)
        )

        coord_samples = self._coords_as_vol[
            :,
            patch_center_el_coords[:, 0],
            patch_center_el_coords[:, 1],
            patch_center_el_coords[:, 2],
        ]
        coord_samples = einops.rearrange(coord_samples, "coord b -> b coord 1 1 1")
        if coord_shifts is not None:
            shifts = einops.rearrange(coord_shifts, "b coord -> b coord 1 1 1")
            coord_samples = coord_samples + shifts
        return coord_samples

    def __iter__(self):
        info = torch.utils.data.get_worker_info()
        num_workers = info.num_workers if info is not None else 1
        id = info.id if info is not None else 0

        for i in range(self.n_batches_per_epoch):
            # Sample locations one batch at a time.
            self.randomize(self.n_samples_per_batch)
            if i % num_workers == id:
                rand_coords = self.get_random_coords_from_rv(self.random_vars)
                yield rand_coords

        self._epoch += 1

In [ ]:
def pretrain_warped_atlas(
    epinr: EPINR,
    max_iters: int,
    subj_data: DWISubjectData,
    params: Box,
    optim: torch.optim.Optimizer,
    batch_size: int,
    pr=print,
) -> dict:
    y = subj_data.suscept_atlas_mm
    assert y.shape[1] == 1
    y_mask = y > 0.0
    # B x 3
    coords = subj_data.b0_scanner_coord_grid[y_mask.squeeze(1)]
    grid_fov = subj_data.b0_fov
    grid_min_coord = subj_data.b0_min_coord
    grid_fov = einops.rearrange(
        grid_fov,
        "b x y z coord -> (b x y z) coord",
        coord=3,
    )
    grid_min_coord = einops.rearrange(
        grid_min_coord,
        "b x y z coord -> (b x y z) coord",
        coord=3,
    )
    # B x 1
    y = y[y_mask].unsqueeze(-1)
    # Append 0s for x and z coordinates since susceptibility is only along y.
    y = torch.cat(
        [
            torch.zeros_like(y),
            y,
            torch.zeros_like(y),
        ],
        dim=-1,
    )

    loss_fn = torch.nn.MSELoss()
    train_losses = dict(epoch=list(), loss=list())

    pr("Pretraining INR to susceptibility atlas warped to subject space...")
    log_epochs = set(
        [1] + np.linspace(1, max_iters + 1, num=10, dtype=int).tolist() + [max_iters]
    )
    epinr.train()
    optim.zero_grad()
    for epoch in range(1, max_iters + 1):
        pred_y = epinr(
            coords,
            grid_fov=grid_fov,
            grid_min_coord=grid_min_coord,
        )
        loss = loss_fn(pred_y, y)
        loss.backward()
        optim.step()
        optim.zero_grad()
        train_losses["epoch"].append(epoch)
        train_losses["loss"].append(loss.detach().item())
        if epoch in log_epochs:
            pr(f"Epoch {epoch}/{max_iters}, Loss: {loss.item():.6f}")

    pr("Pretraining complete.")
    optim.zero_grad()
    epinr.eval()
    with torch.no_grad():
        # Break up the prediction into smaller chunks to avoid OOM errors.
        n_chunks = 16
        b0_coord_grid_chunks = torch.chunk(
            subj_data.b0_scanner_coord_grid, n_chunks, dim=-2
        )
        pred_displacement_field_mm_chunks = list()
        J_chunks = list()
        J_deform_chunks = list()
        for b0_coord_grid_chunk in b0_coord_grid_chunks:
            pred_displacement_field_mm_chunk = epinr(
                x=b0_coord_grid_chunk,
                grid_fov=subj_data.b0_fov,
                grid_min_coord=subj_data.b0_min_coord,
            )
            pred_displacement_field_mm_chunks.append(pred_displacement_field_mm_chunk)
            J_chunk = jacobian_matrix_batched(
                pred_displacement_net=epinr,
                x_coords=b0_coord_grid_chunk,
                grid_fov=subj_data.b0_fov,
                grid_min_coord=subj_data.b0_min_coord,
                jac_of_warp_type="displacement",
            )
            J_chunks.append(J_chunk)
            J_deform_chunk = jacobian_matrix_batched(
                pred_displacement_net=epinr,
                x_coords=b0_coord_grid_chunk,
                grid_fov=subj_data.b0_fov,
                grid_min_coord=subj_data.b0_min_coord,
                jac_of_warp_type="deformation",
            )
            J_deform_chunks.append(J_deform_chunk)
        pred_displacement_field_mm = torch.cat(
            pred_displacement_field_mm_chunks, dim=-2
        )
        J = torch.cat(J_chunks, dim=-3)
        det_J_ap = 1 + J[:, None, ..., 1, 1]
        # Sample the moving b0 volume with the predicted displacement field.
        moving_b0_sample = det_J_ap * mrinr.grid_resample(
            subj_data.b0.expand(
                subj_data.b0_scanner_coord_grid.shape[0], -1, -1, -1, -1
            ),
            affine_x_el2coords=subj_data.b0_affine,
            sample_coords=subj_data.b0_scanner_coord_grid + pred_displacement_field_mm,
            **params.resample_kwargs,
        )

        # Sample the fixed volume without any affine refinement.
        fixed_t1w_sample = mrinr.grid_resample(
            subj_data.t1w.expand(
                subj_data.b0_scanner_coord_grid.shape[0], -1, -1, -1, -1
            ),
            affine_x_el2coords=subj_data.t1w_affine,
            sample_coords=subj_data.b0_scanner_coord_grid,
            **params.resample_kwargs,
        )
        plt.clf()
        plt.figure(dpi=150)
        mrinr.viz.plot_1_ch_multi_slice_compare_vols(
            subj_data.b0,
            moving_b0_sample,
            fixed_t1w_sample,
            subj_data.suscept_atlas_mm,
            pred_displacement_field_mm[..., 1],
            det_J_ap,
            colorbars="rows",
            vol_labels=[
                "Input Distorted b0",
                "Pred. b0 Undist.",
                "T1w",
                "Suscept. Atlas mm",
                "Pred. Displ. mm Y",
                "Continuous Det. J",
            ],
            axial_slice_idx=(0.35, 0.5),
            saggital_slice_idx=0.45,
        )
        plt.show()

    return dict(
        epinr=epinr,
        optim=optim,
        train_losses=train_losses,
    )


def pretrain_fixed_moving_inrs(
    fixed: mrinr.typing.Volume,
    fixed_affine: mrinr.typing.HomogeneousAffine3D,
    moving: mrinr.typing.Volume,
    moving_affine: mrinr.typing.HomogeneousAffine3D,
    moving_coord_grid: mrinr.typing.CoordGrid3D,
    grid_fov: torch.Tensor,
    grid_min_coord: torch.Tensor,
    max_iters: int,
    lambda_recon: float,
    lambda_intermediate_sync: float,
    inr_kwargs: dict,
    optim_kwargs: dict,
    mi_kwargs: dict,
    batch_size: int,
    patch_size: tuple[int, int, int],
    device,
    optim_class=torch.optim.AdamW,
) -> dict:
    assert fixed.shape[1] == moving.shape[1] == 1

    # Random patch dataset.
    ######
    to_be_patched_coord_grid = mrinr.nn.coords_as_channels(
        moving_coord_grid.squeeze(0).cpu(), has_batch_dim=False
    )
    coord_patch_dataset = monai.data.PatchDataset(
        # Data is a shallow copy of the same input coordinate grid, one copy for each batch
        # in the epoch.
        data=[
            {
                "coords": to_be_patched_coord_grid,
                "weight_mask": to_be_patched_coord_grid.new_ones(
                    [1] + list(to_be_patched_coord_grid.shape[1:])
                ),
            }
        ]
        * max_iters,
        patch_func=monai.transforms.RandWeightedCropDict(
            keys="coords",
            w_key="weight_mask",
            spatial_size=patch_size,
            num_samples=batch_size,
        ).set_random_state(seed=1),
        samples_per_image=batch_size,
    )
    #######
    coord_patch_dataset = RandSingleVolCoordSampleDataset(
        coord_grid=moving_coord_grid.squeeze(0).cpu(),
        weight_mask=torch.ones(1, *moving_coord_grid.shape[1:-1]),
        n_samples_per_batch=batch_size,
        n_batches_per_epoch=max_iters,
    )
    #######
    coord_patch_dataloader = monai.data.DataLoader(
        coord_patch_dataset,
        batch_size=None,
        collate_fn=None,
        num_workers=3,
        prefetch_factor=3,
        pin_memory=True,
        shuffle=False,
    )
    ######

    # Create initially identical INRs for fixed and moving volumes.
    fixed_inr = ImageReconINR(**inr_kwargs).to(device)
    moving_inr = ImageReconINR(**inr_kwargs).to(device)
    moving_inr.load_state_dict(fixed_inr.state_dict())

    optim = optim_class(
        [{"params": fixed_inr.parameters()}, {"params": moving_inr.parameters()}],
        **optim_kwargs,
    )

    recon_loss = torch.nn.MSELoss().to(device)
    mi_loss = WeightedNMIParzenLoss(**mi_kwargs, reduction="none").to(device)
    train_losses = dict(epoch=list(), network=list(), metric=list(), loss=list())
    optim.zero_grad()

    print("Pretraining fixed and moving INRs...")
    log_epochs = set(
        [1] + np.linspace(1, max_iters + 1, num=20, dtype=int).tolist() + [max_iters]
    )
    fixed_inr.train()
    moving_inr.train()
    for epoch, coords in enumerate(coord_patch_dataloader, start=1):
        coords = mrinr.nn.channels_as_coords(coords, has_batch_dim=True).to(device)
        coords = einops.rearrange(coords, "b x y z coord -> 1 (b x) y z coord")
        f = mrinr.grid_resample(
            fixed.expand(coords.shape[0], -1, -1, -1, -1),
            affine_x_el2coords=fixed_affine,
            sample_coords=coords,
            mode_or_interpolation="linear",
            padding_mode_or_bound="zeros",
            interp_lib="torch",
        )
        m = mrinr.grid_resample(
            moving.expand(coords.shape[0], -1, -1, -1, -1),
            affine_x_el2coords=moving_affine,
            sample_coords=coords,
            mode_or_interpolation="linear",
            padding_mode_or_bound="zeros",
            interp_lib="torch",
        )

        pred_f, f_intermediates = fixed_inr(
            coords,
            grid_fov=grid_fov,
            grid_min_coord=grid_min_coord,
            return_intermediates=True,
        )
        pred_m, m_intermediates = moving_inr(
            coords,
            grid_fov=grid_fov,
            grid_min_coord=grid_min_coord,
            return_intermediates=True,
        )
        loss_fixed_recon = recon_loss(pred_f, f)
        loss_moving_recon = recon_loss(pred_m, m)
        if lambda_intermediate_sync > 0.0:
            mi_intermediate_loss = list()
            for f_int, m_int in zip(f_intermediates, m_intermediates):
                # Sum over the channel dimension, should be shape (B,)
                mi_intermediate_loss.append(
                    mi_loss(
                        f_int,
                        m_int,
                    ).mean(1)
                )
            mi_intermediate_loss = torch.stack(mi_intermediate_loss, dim=0).mean()
        else:
            mi_intermediate_loss = torch.tensor(0.0).to(device)
        loss_fixed = (lambda_recon * loss_fixed_recon) + (
            lambda_intermediate_sync * mi_intermediate_loss
        )
        loss_moving = (lambda_recon * loss_moving_recon) + (
            lambda_intermediate_sync * mi_intermediate_loss
        )

        loss_fixed.backward(retain_graph=True)
        loss_moving.backward()
        optim.step()
        optim.zero_grad()
        for network_name, metric_name, l in zip(
            ["fixed_inr", "moving_inr"] * 2,
            ["recon"] * 2 + ["intermediate_mi"] * 2,
            [
                lambda_recon * loss_fixed_recon.detach().item(),
                lambda_recon * loss_moving_recon.detach().item(),
                lambda_intermediate_sync * mi_intermediate_loss.detach().item(),
                lambda_intermediate_sync * mi_intermediate_loss.detach().item(),
            ],
        ):
            train_losses["epoch"].append(epoch)
            train_losses["network"].append(network_name)
            train_losses["metric"].append(metric_name)
            train_losses["loss"].append(l)

        if epoch in log_epochs:
            print(
                f"Epoch {epoch}/{max_iters},"
                f"Fixed recon loss: {lambda_recon * loss_fixed_recon.detach().item():.6f}, "
                f"Moving recon loss: {lambda_recon * loss_moving_recon.detach().item():.6f}, "
                f"Intermediate MI loss: {lambda_intermediate_sync * mi_intermediate_loss.detach().item():.6f}"
            )

    optim.zero_grad()
    print("Pretraining complete.")

    n_chunks = 16
    moving_coord_grid_chunks = torch.chunk(moving_coord_grid, n_chunks, dim=-2)
    pred_fixed_chunks = list()
    pred_moving_chunks = list()
    with torch.no_grad():
        for moving_coord_grid_chunk in moving_coord_grid_chunks:
            pred_f_chunk = fixed_inr(
                moving_coord_grid_chunk,
                grid_fov=grid_fov,
                grid_min_coord=grid_min_coord,
            )
            pred_fixed_chunks.append(pred_f_chunk.cpu())
            pred_m_chunk = moving_inr(
                moving_coord_grid_chunk,
                grid_fov=grid_fov,
                grid_min_coord=grid_min_coord,
            )
            pred_moving_chunks.append(pred_m_chunk.cpu())
    pred_fixed = torch.cat(pred_fixed_chunks, dim=-1)
    pred_moving = torch.cat(pred_moving_chunks, dim=-1)

    plt.clf()
    plt.figure(dpi=250)
    mrinr.viz.plot_1_ch_multi_slice_compare_vols(
        fixed[0],
        pred_fixed,
        moving[0],
        pred_moving,
        vol_labels=[
            "Fixed Volume",
            "Fixed INR Recon",
            "Moving Volume",
            "Moving INR Recon",
        ],
        colorbars="global",
        axial_slice_idx=(0.35, 0.5),
        saggital_slice_idx=0.45,
    )
    plt.show()

    return dict(fixed_inr=fixed_inr, moving_inr=moving_inr, train_losses=train_losses)


def pretrain_sequential_fixed_moving_inrs(
    fixed: mrinr.typing.Volume,
    fixed_affine: mrinr.typing.HomogeneousAffine3D,
    moving: mrinr.typing.Volume,
    moving_affine: mrinr.typing.HomogeneousAffine3D,
    moving_coord_grid: mrinr.typing.CoordGrid3D,
    grid_fov: torch.Tensor,
    grid_min_coord: torch.Tensor,
    fixed_recon_iters: int,
    moving_align_iters: int,
    lambda_recon: float,
    lambda_intermediate_sync: float,
    inr_kwargs: dict,
    optim_kwargs: dict,
    fixed_lr_scheduler_kwargs: dict,
    moving_lr_scheduler_kwargs: dict,
    mi_kwargs: dict,
    batch_size: int,
    p_sample_coord_shift: float,
    device,
    rand_sampler_seed: int = 1,
    optim_class=torch.optim.AdamW,
) -> dict:
    assert fixed.shape[1] == moving.shape[1] == 1

    moving_spacing = mrinr.coords.spacing(moving_affine).flatten().cpu().numpy()
    coord_patch_dataset = RandSingleVolCoordSampleDataset(
        coord_grid=moving_coord_grid.squeeze(0).cpu(),
        weight_mask=torch.ones(1, *moving_coord_grid.shape[1:-1]),
        n_samples_per_batch=batch_size,
        n_batches_per_epoch=fixed_recon_iters,
        p_rand_coord_shift=p_sample_coord_shift,
        rand_shift_ranges=(moving_spacing / 2).tolist(),
        seed=rand_sampler_seed,
    )
    coord_patch_dataloader = monai.data.DataLoader(
        coord_patch_dataset,
        batch_size=None,
        collate_fn=None,
        num_workers=3,
        prefetch_factor=3,
        pin_memory=True,
        shuffle=False,
    )
    ######
    # 1. Train the fixed INR on reconstruction alone.
    # Create initially identical INRs for fixed and moving volumes.
    fixed_inr = ImageReconINR(**inr_kwargs).to(device)
    #!TESTING
    moving_inr = ImageReconINR(**inr_kwargs).to(device)
    # Copy weights from fixed INR to initialize.
    #!
    moving_inr.load_state_dict(fixed_inr.state_dict())
    optim = optim_class(fixed_inr.parameters(), **optim_kwargs)

    recon_loss = torch.nn.MSELoss().to(device)

    optim.zero_grad()

    print("Training fixed INR reconstruction only...")
    log_epochs = set(
        [1]
        + np.linspace(1, fixed_recon_iters + 1, num=20, dtype=int).tolist()
        + [fixed_recon_iters]
    )
    fixed_inr_loss = dict(epoch=list(), network=list(), metric=list(), loss=list())

    fixed_inr.train()
    for epoch, coords in enumerate(coord_patch_dataloader, start=1):
        coords = mrinr.nn.channels_as_coords(coords, has_batch_dim=True).to(device)
        coords = einops.rearrange(coords, "b x y z coord -> 1 (b x) y z coord")
        f = mrinr.grid_resample(
            fixed.expand(coords.shape[0], -1, -1, -1, -1),
            affine_x_el2coords=fixed_affine,
            sample_coords=coords,
            mode_or_interpolation="linear",
            padding_mode_or_bound="zeros",
            interp_lib="torch",
        )
        pred_f = fixed_inr(
            coords,
            grid_fov=grid_fov,
            grid_min_coord=grid_min_coord,
            return_intermediates=False,
        )
        loss = recon_loss(pred_f, f)
        loss.backward()
        optim.step()
        optim.zero_grad()
        fixed_inr_loss["epoch"].append(epoch)
        fixed_inr_loss["network"].append("fixed_inr")
        fixed_inr_loss["metric"].append("recon")
        fixed_inr_loss["loss"].append(loss.detach().item())

        if epoch in log_epochs:
            print(
                f"Epoch {epoch}/{fixed_recon_iters},"
                f"Fixed recon loss: {loss.detach().item():.6f}, "
            )
    optim.zero_grad()
    print("Fixed INR pretraining complete.")

    # 2. Train the moving INR with both reconstruction and feature alignment losses.
    coord_patch_dataset = RandSingleVolCoordSampleDataset(
        coord_grid=moving_coord_grid.squeeze(0).cpu(),
        weight_mask=torch.ones(1, *moving_coord_grid.shape[1:-1]),
        n_samples_per_batch=batch_size,
        n_batches_per_epoch=moving_align_iters,
        p_rand_coord_shift=p_sample_coord_shift,
        rand_shift_ranges=(moving_spacing / 2).tolist(),
        seed=rand_sampler_seed,
    )
    coord_patch_dataloader = monai.data.DataLoader(
        coord_patch_dataset,
        batch_size=None,
        collate_fn=None,
        num_workers=3,
        prefetch_factor=3,
        pin_memory=True,
        shuffle=False,
    )

    #! moving_inr = ImageReconINR(**inr_kwargs).to(device)
    # # Copy weights from fixed INR to initialize.
    #! moving_inr.load_state_dict(fixed_inr.state_dict())
    optim = optim_class(moving_inr.parameters(), **optim_kwargs)
    mi_loss = WeightedNMIParzenLoss(**mi_kwargs, reduction="none").to(device)
    fixed_inr.eval()
    moving_inr.train()
    moving_inr_loss = dict(epoch=list(), network=list(), metric=list(), loss=list())
    print("Pretraining fixed and moving INRs...")
    log_epochs = set(
        [1]
        + np.linspace(1, moving_align_iters + 1, num=20, dtype=int).tolist()
        + [moving_align_iters]
    )
    for epoch, coords in enumerate(coord_patch_dataloader, start=1):
        coords = mrinr.nn.channels_as_coords(coords, has_batch_dim=True).to(device)
        coords = einops.rearrange(coords, "b x y z coord -> 1 (b x) y z coord")

        m = mrinr.grid_resample(
            moving.expand(coords.shape[0], -1, -1, -1, -1),
            affine_x_el2coords=moving_affine,
            sample_coords=coords,
            mode_or_interpolation="linear",
            padding_mode_or_bound="zeros",
            interp_lib="torch",
        )
        pred_m, m_intermediates = moving_inr(
            coords,
            grid_fov=grid_fov,
            grid_min_coord=grid_min_coord,
            return_intermediates=True,
        )

        if lambda_intermediate_sync > 0.0:
            with torch.no_grad():
                _, f_intermediates = fixed_inr(
                    coords,
                    grid_fov=grid_fov,
                    grid_min_coord=grid_min_coord,
                    return_intermediates=True,
                )
            feature_align_mi = list()
            for f_int, m_int in zip(f_intermediates, m_intermediates):
                # Sum over the channel dimension, should be shape (B,)
                feature_align_mi.append(
                    mi_loss(
                        f_int,
                        m_int,
                    ).mean(1)
                )
            feature_align_mi = torch.stack(feature_align_mi, dim=0).mean()
        else:
            feature_align_mi = torch.tensor(0.0).to(device)

        loss_recon = recon_loss(pred_m, m)
        loss = (lambda_recon * loss_recon) + (
            lambda_intermediate_sync * feature_align_mi
        )
        loss.backward()

        optim.step()
        optim.zero_grad()

        for metric_name, l in zip(
            ["recon", "feature_align_mi"],
            [
                lambda_recon * loss_recon.detach().item(),
                lambda_intermediate_sync * feature_align_mi.detach().item(),
            ],
        ):
            moving_inr_loss["epoch"].append(epoch)
            moving_inr_loss["network"].append("moving_inr")
            moving_inr_loss["metric"].append(metric_name)
            moving_inr_loss["loss"].append(l)

        if epoch in log_epochs:
            print(
                f"Epoch {epoch}/{moving_align_iters},"
                f"Moving recon loss: {lambda_recon * loss_recon.detach().item():.6f}, "
                f"Intermediate MI loss: {lambda_intermediate_sync * feature_align_mi.detach().item():.6f}"
            )

    optim.zero_grad()
    print("Pretraining complete.")

    n_chunks = 16
    moving_coord_grid_chunks = torch.chunk(moving_coord_grid, n_chunks, dim=-2)
    pred_fixed_chunks = list()
    pred_moving_chunks = list()
    with torch.no_grad():
        for moving_coord_grid_chunk in moving_coord_grid_chunks:
            pred_f_chunk = fixed_inr(
                moving_coord_grid_chunk,
                grid_fov=grid_fov,
                grid_min_coord=grid_min_coord,
            )
            pred_fixed_chunks.append(pred_f_chunk.cpu())
            pred_m_chunk = moving_inr(
                moving_coord_grid_chunk,
                grid_fov=grid_fov,
                grid_min_coord=grid_min_coord,
            )
            pred_moving_chunks.append(pred_m_chunk.cpu())
    pred_fixed = torch.cat(pred_fixed_chunks, dim=-1)
    pred_moving = torch.cat(pred_moving_chunks, dim=-1)

    plt.clf()
    plt.figure(dpi=250)
    mrinr.viz.plot_1_ch_multi_slice_compare_vols(
        fixed[0],
        pred_fixed,
        moving[0],
        pred_moving,
        vol_labels=[
            "Fixed Volume",
            "Fixed INR Recon",
            "Moving Volume",
            "Moving INR Recon",
        ],
        colorbars="global",
        axial_slice_idx=(0.35, 0.5),
        saggital_slice_idx=0.45,
    )
    plt.show()

    return dict(
        fixed_inr=fixed_inr,
        moving_inr=moving_inr,
        fixed_inr_loss=fixed_inr_loss,
        moving_inr_loss=moving_inr_loss,
    )

In [ ]:
_dataset_dirs = {
    "mica_mics": Path("~/mnt/magpie/outputs/mica_mics/derivatives/epinr_fmap_learning")
    .expanduser()
    .resolve(),
    "vcu_ms": Path("~/mnt/magpie/outputs/vcu_ms_epinr/derivatives/epinr_fmap_learning")
    .expanduser()
    .resolve(),
}
_dataset_table_f = Path("data_tables/merged_dmri_dataset_table.csv").resolve()

## Susceptibility Distortion Field Reconstruction

### Whole-Volume Batches

In [ ]:
# Experiment params
params = Box(
    dict(
        dataset_dirs=_dataset_dirs,
        dataset_table_f=_dataset_table_f,
        result_base_dir=Path("~/mnt/magpie/outputs/results/epinr/dmri/epinr/")
        .expanduser()
        .resolve(),
        # epochs=75,
        # n_repeats_per_epoch=40,
        epochs=5,
        n_repeats_per_epoch=5,
        effective_batch_size=8,
        batch_size=2,
        patch_size=(32, 32, 32),
        winsorize_quantiles=(0.01, 0.99),
        feature_range=(0.0, 1.0),
        mask_blur_sigma_mm=2.0,
        mask_blur_truncate=5.0,
        # INR model params
        hidden_features=256,
        num_layers=5,
        omega=1.0,
        omega_0=30.0,
        c=6.0,
        # Learnable affine transform params.
        # Moving volume non-rigid affine components.
        non_rigid_affine_layer=False,
        non_rigid_affine_nonzero_thresh=None,
        # Fixed volume, refined with rigid affine only.
        refine_fixed_rigid_affine=True,
        # Percentage of training epochs that will optimize the fixed volume alignment.
        # After this percentage of epochs, the fixed volume rigid affine will be
        # frozen.
        refine_fixed_rigid_affine_epoch_percent=0.5,
        # Positional encoding params
        m_num_freqs=None,
        pos_enc_sigma=None,
        pos_enc_type=None,
        # m_num_freqs=512,
        # pos_enc_sigma=0.5,
        # pos_enc_type="gaussian",
        # Training function params
        # Pretrain to warped atlas.
        pretrain_to_atlas=False,
        pretrain_epochs=None,
        optim_lr=0.0005,
        optim_weight_decay=0.01,
        optim_betas=(0.9, 0.999),
        # Linear LR scheduler end scale factor.
        lr_scheduler_end_factor=0.01,
        # Percentage of training epochs for constant warmup and constant cooldown
        # phases, both before and after the linear decay phase. Each warmup and cooldown
        # phase lasts for this percentage, so the linear decay phase lasts for
        # 1 - (2 * this percentage).
        lr_scheduler_warmup_cooldown_percent=0.15,
        enable_jac_mod_in_training=True,
        mse_sim_weight=0.0,
        mi_sim_weight=1.0,
        mi_sim_kwargs=dict(
            num_bins=32, sigma_ratio=0.5, norm_mi=False, norm_images=True
        ),
        ncc_sim_weight=0.0,
        lncc_sim_weight=0.0,
        lncc_sim_kwargs=dict(kernel_size=9, kernel_type="rectangular"),
        laplacian_ncc_sim_weight=0.5,
        laplacian_mse_sim_weight=0.0,
        laplacian_sigma_low=0.5,
        laplacian_truncate=2.0,
        jac_fro_reg_weight=1.5,
        bending_energy_reg_weight=2.0,
        resample_kwargs=dict(
            mode_or_interpolation="linear",
            padding_mode_or_bound="zeros",
            interp_lib="torch",
        ),
        val_mi_sim_kwargs=dict(kernel_type="gaussian", num_bins=32, sigma_ratio=0.5),
    ),
)
params.result_base_dir.mkdir(parents=True, exist_ok=True)
grad_accumulate_steps = params.effective_batch_size / params.batch_size
assert grad_accumulate_steps.is_integer(), (
    f"Effective batch size {params.effective_batch_size} must be "
    f"a multiple of batch size {params.batch_size}."
)
grad_accumulate_steps = int(grad_accumulate_steps)
n_batches_per_epoch = params.n_repeats_per_epoch * grad_accumulate_steps
viz_validation_every: int = max(params.epochs // 10, 1)  # epochs

In [ ]:
# #!
# pretrain_results = pretrain_sequential_fixed_moving_inrs(
#     fixed=subj_data.t1w,
#     fixed_affine=subj_data.t1w_affine,
#     moving=subj_data.b0,
#     moving_affine=subj_data.b0_affine,
#     moving_coord_grid=subj_data.b0_scanner_coord_grid,
#     grid_fov=subj_data.b0_fov,
#     grid_min_coord=subj_data.b0_min_coord,
#     fixed_recon_iters=100_000,
#     moving_align_iters=100_000,
#     lambda_recon=1.0,
#     lambda_intermediate_sync=0.0,
#     inr_kwargs=dict(
#         hidden_features=256,
#         num_layers=5,
#         out_features=1,
#         m_num_freqs=1024,
#         sigma=5.0,
#         pos_enc="gaussian",
#     ),
#     optim_kwargs=dict(
#         lr=0.00001,
#         weight_decay=params.optim_weight_decay,
#         betas=params.optim_betas,
#     ),
#     mi_kwargs=dict(num_bins=16, sigma_ratio=0.5, norm_mi=True, norm_images=False),
#     batch_size=20_000,
#     p_sample_coord_shift=0.5,
#     device=device,
# )
# #!
# pretrain_fixed_recon_losses = pd.DataFrame(pretrain_results["fixed_inr_loss"])
# pretrain_moving_losses = pd.DataFrame(pretrain_results["moving_inr_loss"])
# pretrain_losses = pd.concat(
#     [pretrain_fixed_recon_losses, pretrain_moving_losses], axis=0
# )
# plt.clf()
# plt.figure(dpi=150)
# fg = sns.relplot(
#     data=pretrain_losses,
#     x="epoch",
#     y="loss",
#     hue="network",
#     col="metric",
#     kind="line",
#     alpha=0.8,
#     facet_kws={"sharey": False},
# )
# fg.axes[0, 0].set_yscale("log")

In [ ]:
def validation_step(
    epinr: EPINR,
    subj_data: DWISubjectData,
    params: Box,
    epoch: int,
    step: int,
    val_score_fns: dict,
    b0_spacing: torch.Tensor,
    viz_validation_every: int,
    pr=print,
) -> dict:
    # Convenience functions for resampling volumes.
    sample_t1w = lambda c: mrinr.grid_resample(
        subj_data.t1w.expand(c.shape[0], -1, -1, -1, -1),
        affine_x_el2coords=subj_data.t1w_affine,
        sample_coords=c,
        **params.resample_kwargs,
    )
    sample_b0 = lambda c: mrinr.grid_resample(
        subj_data.b0.expand(c.shape[0], -1, -1, -1, -1),
        affine_x_el2coords=subj_data.b0_affine,
        sample_coords=c,
        **params.resample_kwargs,
    )

    # Break up the prediction into smaller chunks to avoid OOM errors.
    n_chunks = 16
    b0_coord_grid_chunks = torch.chunk(
        subj_data.b0_scanner_coord_grid, n_chunks, dim=-2
    )
    pred_displacement_field_mm_chunks = list()
    J_chunks = list()
    J_deform_chunks = list()
    for b0_coord_grid_chunk in b0_coord_grid_chunks:
        pred_displacement_field_mm_chunk = epinr(
            x=b0_coord_grid_chunk,
            grid_fov=subj_data.b0_fov,
            grid_min_coord=subj_data.b0_min_coord,
        )
        pred_displacement_field_mm_chunks.append(pred_displacement_field_mm_chunk)
        J_chunk = jacobian_matrix_batched(
            pred_displacement_net=epinr,
            x_coords=b0_coord_grid_chunk,
            grid_fov=subj_data.b0_fov,
            grid_min_coord=subj_data.b0_min_coord,
            jac_of_warp_type="displacement",
        )
        J_chunks.append(J_chunk)
        J_deform_chunk = jacobian_matrix_batched(
            pred_displacement_net=epinr,
            x_coords=b0_coord_grid_chunk,
            grid_fov=subj_data.b0_fov,
            grid_min_coord=subj_data.b0_min_coord,
            jac_of_warp_type="deformation",
        )
        J_deform_chunks.append(J_deform_chunk)
    pred_displacement_field_mm = torch.cat(pred_displacement_field_mm_chunks, dim=-2)
    J = torch.cat(J_chunks, dim=-3)
    det_J_ap = 1 + J[:, None, ..., 1, 1]
    J_deform = torch.cat(J_deform_chunks, dim=-3)
    # Sample the moving b0 volume with the predicted displacement field.
    moving_b0_sample = det_J_ap * sample_b0(
        subj_data.b0_scanner_coord_grid + pred_displacement_field_mm
    )

    ## Compute validation scores.
    # Sample the fixed volume without any affine refinement.
    t1w_coord_grid = mrinr.coords.affine_coord_grid(
        subj_data.t1w_affine,
        subj_data.t1w.shape[-3:],
    ).to(subj_data.b0_scanner_coord_grid)
    # Sample T1w at the coordinates of the moving image for quant. comparison.
    fixed_t1w_sample = sample_t1w(subj_data.b0_scanner_coord_grid)
    # Also sample at the full T1w resolution for visualization.
    viz_fixed_t1w_sample = sample_t1w(t1w_coord_grid)
    if epinr.fixed2moving_rigid_affine is not None:
        rigid_refine_fixed_t1w_sample = sample_t1w(
            epinr.apply_fixed_rigid_affine(t1w_coord_grid)
        )
    else:
        rigid_refine_fixed_t1w_sample = None
    # Convert the topup ground truth displacement field to mm units.
    topup_suscept_field_mm = (
        subj_data.topup_displace_hz
        * subj_data.total_readout_time_s
        * b0_spacing[0, 1]
        * (-1 if subj_data.pe_dir == "ap" else 1)
    )[:, 0]
    val_scores_epoch_i = {
        "fixed_moving_mi": val_score_fns["fixed_moving_mi"](
            moving_b0_sample, fixed_t1w_sample
        )
        .cpu()
        .item(),
        "fixed_moving_ncc": val_score_fns["fixed_moving_ncc"](
            moving_b0_sample, fixed_t1w_sample
        )
        .cpu()
        .item(),
        "topup_suscept_mse": val_score_fns["topup_suscept_mse"](
            pred_displacement_field_mm[..., 1], topup_suscept_field_mm
        )
        .cpu()
        .item(),
        "topup_corrected_b0_mse": val_score_fns["topup_corrected_b0_mse"](
            moving_b0_sample, subj_data.topup_corrected_b0
        )
        .cpu()
        .item(),
        "masked_topup_suscept_mse": val_score_fns["masked_topup_suscept_mse"](
            pred_displacement_field_mm[..., 1],
            topup_suscept_field_mm,
            subj_data.b0_mask,
        )
        .cpu()
        .item(),
        "masked_topup_corrected_b0_mse": val_score_fns["masked_topup_corrected_b0_mse"](
            moving_b0_sample, subj_data.topup_corrected_b0, subj_data.b0_mask
        )
        .cpu()
        .item(),
        "det_J_def_neg_frac": val_score_fns["det_J_def_neg_frac"](J_deform)
        .cpu()
        .item(),
    }
    pr(f"Validation scores at epoch {epoch}:")
    for metric_name, val_score in val_scores_epoch_i.items():
        # val_scores["epoch"].append(epoch)
        # val_scores["step"].append(step)
        # val_scores["metric"].append(metric_name)
        # val_scores["val"].append(val_score)
        pr(f"{metric_name}: {val_score: 10.9f}", end=" | ")
    pr(flush=True)

    if epinr.nonrigid_affine_layer is not None:
        pr(
            "Nonrigid affine for moving image\n",
            {
                k: v.detach().cpu().numpy()
                for k, v in epinr.nonrigid_affine_layer.named_parameters()
            },
        )
    if epinr.fixed_rigid_affine is not None:
        pr(
            "Rigid affine for fixed image refinement:\n",
            {
                k: v.detach().cpu().numpy()
                for k, v in epinr.fixed_rigid_affine.named_parameters()
            },
        )

    if viz_validation_every and (
        epoch % viz_validation_every == 0 or epoch == params.epochs
    ):
        plt.clf()
        plt.figure(dpi=175)
        vols = list()
        vol_labels = list()
        vols.extend([subj_data.b0, moving_b0_sample, viz_fixed_t1w_sample])
        vol_labels.extend(["In Dist b0", "Pred b0 Undist.", "T1"])
        if rigid_refine_fixed_t1w_sample is not None:
            vols.append(rigid_refine_fixed_t1w_sample)
            vol_labels.append("Refine T1")
        vols.extend(
            [
                torch.abs(subj_data.b0 - moving_b0_sample),
                subj_data.topup_corrected_b0,
                torch.abs(subj_data.topup_corrected_b0 - moving_b0_sample),
                subj_data.topup_displace_hz
                * subj_data.total_readout_time_s
                * b0_spacing[0, 1]
                * (-1 if subj_data.pe_dir == "ap" else 1),
                pred_displacement_field_mm[..., 1],
                det_J_ap,
            ]
        )
        vol_labels.extend(
            [
                "b0 Dist - Undist",
                "Topup b0",
                "Topup - Pred",
                "Topup Disp mm Y",
                "Pred Disp mm Y",
                "Det J",
            ]
        )
        mrinr.viz.plot_1_ch_multi_slice_compare_vols(
            *vols,
            colorbars="rows",
            vol_labels=vol_labels,
            axial_slice_idx=(0.35, 0.5),
            saggital_slice_idx=0.45,
        )
        if epoch != params.epochs:
            plt.show()

    return {
        "val_scores": val_scores_epoch_i,
        "pred_displacement_field_mm": pred_displacement_field_mm[0, ..., 1]
        .cpu()
        .numpy(),
        "pred_undistorted_b0": moving_b0_sample[0, 0].cpu().numpy(),
    }

In [ ]:
# -------------------------------------------------------------------------
FULL_EXPERIMENT = True

#### Training loop.
dataset_table = pd.read_csv(params.dataset_table_f, index_col=None, comment="#")
if FULL_EXPERIMENT:
    dataset_table_rows = list(range(len(dataset_table)))
    # Shuffle subject order to get better estimate of generalization performance, earlier.
    dataset_table_rows = np.random.shuffle(list(dataset_table_rows))
    # dataset_table_rows = [45, 56]  #!
    timestamp_str = mrinr.utils.timestamp_now()
    experiment_label = "epinr_dmri"
    # New experiment name.
    experiment_name = f"{timestamp_str}_{experiment_label}"
    # Fixed experiment name.
    # experiment_name = ""
    final_result_dir = params.result_base_dir / experiment_name
    experiment_result_dir = final_result_dir
    # Fixed result dir.
    # tmp_result_dir_name = final_result_dir.parent / ""
    # Random tmp result dir.
    tmp_result_dir_name = tempfile.mkdtemp(
        dir=final_result_dir.parent,
        prefix=f"tmp-{timestamp_str}-",
        suffix=f"__{final_result_dir.name}",
    )
    tmp_result_dir = Path(tmp_result_dir_name)
    result_dir = tmp_result_dir
    # Save out parameters used for the experiment.
    p = copy.deepcopy(params.to_dict())
    with open(result_dir / "experiment_params.yaml", "w") as f:
        yaml.dump(params.to_dict(), f)
else:
    dataset_table_rows = [45]
    result_dir = Path("tmp").resolve()
    final_result_dir = None
    experiment_result_dir = None

for dataset_table_row in dataset_table_rows:
    subj_table = dataset_table.iloc[dataset_table_row : (dataset_table_row + 1)]
    print(f"Starting subject {subj_table['subj_id'].values.item()}...")

    # Handle random seeding.
    if FULL_EXPERIMENT:
        random_seed = mrinr.utils.create_subj_rng_seed(
            base_rng_seed=1, subj_id=subj_table["subj_id"].values.item()
        ) % (2**32 - 1)
    else:
        random_seed = np.random.randint(0, 2**32 - 1)
        # random_seed = 4221131316
    torch.random.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    patch_select_seed = np.random.randint(0, 2**32 - 1)

    subj_data = load_dwi_subject_data(
        dataset_table=subj_table,
        dataset_dirs=params.dataset_dirs,
        device=device,
    )[0]

    # Create a subject-specific result directory.
    if FULL_EXPERIMENT:
        subj_result_dir = (
            result_dir
            / f"{subj_data.dataset_name}_{subj_data.subj_id}_dir-{subj_data.pe_dir}"
        )
        subj_result_dir.mkdir(parents=True, exist_ok=True)
        completed_indicator_f = subj_result_dir / "epinr_final_model.pt"
        if completed_indicator_f.exists():
            print(
                f"Subject {subj_data.subj_id} already completed,"
                " skipping to next subject..."
            )
            continue
        log_f = subj_result_dir / "log.txt"
        if log_f.exists():
            log_mtime_delta = time.time() - os.path.getmtime(log_f)
            if log_mtime_delta < 30.0:
                print(
                    f"Subject {subj_data.subj_id} is currently running,"
                    " skipping to next subject..."
                )
                continue
            log_f.unlink(missing_ok=True)
        pr = partial(mrinr.utils.tee, file=log_f)
        pr(f"Subject row:\n{dataset_table_row}")
    else:
        subj_result_dir = result_dir
        pr = print

    pr(f"Random seed: {random_seed}")
    pr(f"Loaded subject {subj_data.dataset_name}, {subj_data.subj_id}.")
    # Store predicted displacement fields and undistorted b0 volumes at each validation
    # step.
    val_pred_displacement_fields_f = (
        subj_result_dir / "val_pred_displacement_fields.nii.gz"
    )
    val_undistorted_b0s_f = subj_result_dir / "val_undistorted_b0s.nii.gz"
    final_val_fig_f = subj_result_dir / "final_validation_result.png"
    train_scores_f = subj_result_dir / "train_scores.csv"
    val_scores_f = subj_result_dir / "val_scores.csv"
    train_loss_plot_f = subj_result_dir / "train_loss_plot.png"
    val_scores_plot_f = subj_result_dir / "val_loss_plot.png"
    val_pred_displacement_fields_list = list()
    val_pred_undistorted_b0s_list = list()

    # Prep volumes for training.
    train_vols = prep_vols(
        b0=subj_data.b0,
        b0_mask=subj_data.b0_mask,
        b0_affine=subj_data.b0_affine,
        t1w=subj_data.t1w,
        t1w_mask=subj_data.t1w_mask,
        t1w_affine=subj_data.t1w_affine,
        scale_kwargs=dict(
            winsorize_quantiles=params.winsorize_quantiles,
            feature_range=params.feature_range,
        ),
        blur_mask_kwargs=dict(
            sigma_mm=params.mask_blur_sigma_mm,
            truncate=params.mask_blur_truncate,
        ),
    )
    subj_data.b0 = train_vols["b0"]
    subj_data.t1w = train_vols["t1w"]
    b0_weight_mask = train_vols["b0_mask"]
    t1w_weight_mask = train_vols["t1w_mask"]
    subj_data.topup_corrected_b0 = scale_vol(
        subj_data.topup_corrected_b0,
        winsorize_quantiles=params.winsorize_quantiles,
        feature_range=params.feature_range,
    )

    # Append a batch dimension to all Tensors.
    data_d = dict()
    for k, v in dataclasses.asdict(subj_data).items():
        if isinstance(v, torch.Tensor):
            if "fov" in k:
                v = einops.rearrange(v, "ndim -> 1 1 1 ndim")
            elif "min_coord" in k:
                v = einops.rearrange(v, "ndim -> 1 1 1 ndim")
            data_d[k] = v.unsqueeze(0)
        else:
            data_d[k] = v
    subj_data = DWISubjectData(**data_d)
    b0_spacing = mrinr.coords.spacing(subj_data.b0_affine)
    b0_weight_mask.unsqueeze_(0)
    t1w_weight_mask.unsqueeze_(0)
    # 1 / 0

    # Convenience functions for resampling volumes.
    sample_t1w = lambda c: mrinr.grid_resample(
        subj_data.t1w.expand(c.shape[0], -1, -1, -1, -1),
        affine_x_el2coords=subj_data.t1w_affine,
        sample_coords=c,
        **params.resample_kwargs,
    )
    sample_b0 = lambda c: mrinr.grid_resample(
        subj_data.b0.expand(c.shape[0], -1, -1, -1, -1),
        affine_x_el2coords=subj_data.b0_affine,
        sample_coords=c,
        **params.resample_kwargs,
    )
    sample_t1w_weight_mask = lambda c: mrinr.grid_resample(
        t1w_weight_mask.expand(c.shape[0], -1, -1, -1, -1),
        affine_x_el2coords=subj_data.t1w_affine,
        sample_coords=c,
        **params.resample_kwargs,
    )
    sample_b0_weight_mask = lambda c: mrinr.grid_resample(
        b0_weight_mask.expand(c.shape[0], -1, -1, -1, -1),
        affine_x_el2coords=subj_data.b0_affine,
        sample_coords=c,
        **params.resample_kwargs,
    )

    # Construct a dataset and dataloader to split up the coordinate grids into patches, to
    # save memory during training.
    # The dataset needs a non-batched volume with channel dimensions (not coordinate
    # dimensions) on the cpu. So, move the coordinate grid to the cpu, remove the batch
    # dim, and convert the coord dim to a channel dim.
    to_be_patched_coord_grid = mrinr.nn.coords_as_channels(
        subj_data.b0_scanner_coord_grid.squeeze(0).cpu(), has_batch_dim=False
    )
    # Minibatch of smaller patches that are randomly drawn from the coordinate grid.
    # Combine the b0 and t1w weight masks for sampling.
    combined_b0_t1w_weight_mask = torch.maximum(
        sample_b0_weight_mask(subj_data.b0_scanner_coord_grid),
        sample_t1w_weight_mask(subj_data.b0_scanner_coord_grid),
    )
    coord_patch_dataset = monai.data.PatchDataset(
        # Data is a shallow copy of the same input coordinate grid, one copy for each batch
        # in the epoch.
        data=[
            {
                "coords": to_be_patched_coord_grid,
                "weight_mask": combined_b0_t1w_weight_mask.squeeze(0).cpu(),
            }
        ]
        * n_batches_per_epoch,
        patch_func=monai.transforms.RandWeightedCropDict(
            keys="coords",
            w_key="weight_mask",
            spatial_size=params.patch_size,
            num_samples=params.batch_size,
        ).set_random_state(seed=patch_select_seed),
        samples_per_image=params.batch_size,
    )
    ######
    coord_patch_dataloader = monai.data.DataLoader(
        coord_patch_dataset,
        batch_size=params.batch_size,
        num_workers=4,
        prefetch_factor=3,
        pin_memory=True,
        shuffle=False,
    )

    # Instantiate the EPINR model.
    epinr = EPINR(
        hidden_features=params.hidden_features,
        num_layers=params.num_layers,
        pe_dir=subj_data.pe_dir,
        non_rigid_affine=params.non_rigid_affine_layer,
        non_rigid_affine_nonzero_thresh=params.non_rigid_affine_nonzero_thresh,
        sigma=params.pos_enc_sigma,
        m_num_freqs=params.m_num_freqs,
        pos_enc=params.pos_enc_type,
        refine_fixed_rigid_affine=params.refine_fixed_rigid_affine,
        omega=params.omega,
        omega_0=params.omega_0,
        c=params.c,
    )
    epinr.to(device=device)
    epinr.train()
    pr("Model summary:")
    pr(epinr)
    # Instantiate the optimizer.
    optim = torch.optim.AdamW(
        [{"params": epinr.network_parameters()}]
        + [{"params": epinr.fixed2moving_rigid_affine_parameters()}]
        if params.refine_fixed_rigid_affine
        else list(),
        lr=params.optim_lr / grad_accumulate_steps,
        betas=params.optim_betas,
        weight_decay=params.optim_weight_decay,
        fused=True,
    )
    pr("Optimizer:")
    pr(optim)
    optim.zero_grad()
    warm_cooldown_epochs = math.ceil(
        params.epochs * params.lr_scheduler_warmup_cooldown_percent
    )
    lin_decay_lr_epochs = params.epochs - (2 * warm_cooldown_epochs)
    lr_scheduler = torch.optim.lr_scheduler.SequentialLR(
        optim,
        [
            torch.optim.lr_scheduler.ConstantLR(
                optim, factor=1.0, total_iters=warm_cooldown_epochs
            ),
            torch.optim.lr_scheduler.LinearLR(
                optim,
                start_factor=1.0,
                end_factor=params.lr_scheduler_end_factor,
                total_iters=lin_decay_lr_epochs,
            ),
            torch.optim.lr_scheduler.ConstantLR(
                optim,
                factor=params.lr_scheduler_end_factor,
                total_iters=warm_cooldown_epochs + 1,
            ),
        ],
        milestones=[warm_cooldown_epochs, lin_decay_lr_epochs + warm_cooldown_epochs],
    )
    pr("LR Scheduler milestones:")
    pr(lr_scheduler._milestones)
    if params.refine_fixed_rigid_affine:
        stop_rigid_aff_epoch = math.ceil(
            params.refine_fixed_rigid_affine_epoch_percent * params.epochs
        )
        pr(f"Will stop refining fixed rigid affine at epoch {stop_rigid_aff_epoch}.")
    else:
        stop_rigid_aff_epoch = math.inf

    # Loss functions.
    mse_loss = torch.nn.MSELoss(reduction="none").to(device)
    mi_loss = WeightedNMIParzenLoss(**params.mi_sim_kwargs, reduction="mean").to(device)
    ncc_loss = NCC(use_mask=False).to(device)
    lncc_loss = monai.losses.LocalNormalizedCrossCorrelationLoss(
        spatial_dims=3, **params.lncc_sim_kwargs, reduction="mean"
    ).to(device)
    laplacian_fn = DoGLaplacian(
        sigma_low=params.laplacian_sigma_low, truncate=params.laplacian_truncate
    ).to(device)
    loss_fn = mrinr.nn.loss.WeightedSumLoss(
        loss_fns={
            "weighted_mse_sim": lambda pred, target, weight_mask: (
                mse_loss(pred, target) * weight_mask
            ).mean(),
            "weighted_mi_sim": mi_loss,
            "weighted_ncc_sim": ncc_loss,
            "weighted_lncc_sim": lncc_loss,
            "laplace_ncc_sim": lambda pred, target: lncc_loss(
                laplacian_fn(pred),
                laplacian_fn(1 - target),  # 1-T1w to invert contrast
                # laplacian_fn(target),  #! do not invert contrast
            ),
            "laplace_mse_sim": lambda pred, target, weight_mask: (
                mse_loss(
                    laplacian_fn(pred),
                    laplacian_fn(1 - target),  # 1-T1w to invert contrast
                    # laplacian_fn(target),  #! do not invert contrast
                )
                * weight_mask
            ).mean(),
            "jac_fro_regularization": jac_displace_fro_regularization,
            "bending_energy_regularization": lambda model,
            c: bending_energy_regularization(
                model,
                x_coords=c,
                grid_fov=subj_data.b0_fov,
                grid_min_coord=subj_data.b0_min_coord,
                pe_dir=subj_data.pe_dir,
                hessian_of_warp_type="deformation",
                compile_hessian=True,
            ),
        },
        loss_weights=[
            params.mse_sim_weight,
            params.mi_sim_weight,
            params.ncc_sim_weight,
            params.lncc_sim_weight,
            params.laplacian_ncc_sim_weight,
            params.laplacian_mse_sim_weight,
            params.jac_fro_reg_weight,
            params.bending_energy_reg_weight,
        ],
    ).to(device)
    val_score_fns = {
        "fixed_moving_mi": monai.losses.GlobalMutualInformationLoss(
            **params.val_mi_sim_kwargs, reduction="mean"
        ).to(device),
        "fixed_moving_ncc": monai.losses.LocalNormalizedCrossCorrelationLoss(
            spatial_dims=3, **params.lncc_sim_kwargs, reduction="mean"
        ).to(device),
        "topup_suscept_mse": torch.nn.MSELoss(reduction="mean").to(device),
        "topup_corrected_b0_mse": torch.nn.MSELoss(reduction="mean").to(device),
        "masked_topup_suscept_mse": mrinr.nn.loss.WeightedMaskLoss(
            "mse", reduction="mean"
        ).to(device),
        "masked_topup_corrected_b0_mse": mrinr.nn.loss.WeightedMaskLoss(
            "mse", reduction="mean"
        ).to(device),
        "det_J_def_neg_frac": lambda J_deform: (
            (torch.linalg.det(J_deform) < 0.0).sum()
        )
        / J_deform[..., 0, 0].numel(),
    }

    ### Training loop
    if params.pretrain_to_atlas:
        pretrain_optim = torch.optim.AdamW(
            epinr.parameters(),
            lr=params.optim_lr,
            betas=params.optim_betas,
            weight_decay=params.optim_weight_decay,
            fused=True,
        )
        pretrain_results = pretrain_warped_atlas(
            epinr=epinr,
            max_iters=params.pretrain_epochs,
            subj_data=subj_data,
            params=params,
            optim=pretrain_optim,
            batch_size=None,
        )
        epinr = pretrain_results["epinr"]
        pretrain_loss = pretrain_results["train_losses"]
        del pretrain_optim

    train_loss = {
        "epoch": list(),
        "step": list(),
        "loss_term": list(),
        "val": list(),
        "weighted": list(),
    }
    accumulate_train_loss = collections.defaultdict(list)
    val_scores = {"epoch": list(), "step": list(), "metric": list(), "val": list()}
    step = 1
    batch_step = 1
    for epoch in range(1, params.epochs + 1):
        optim.zero_grad()
        epinr.train()
        for i_epoch_batch, batch_dict in enumerate(coord_patch_dataloader):
            if ((i_epoch_batch + 1) % grad_accumulate_steps != 0) and (
                i_epoch_batch < (n_batches_per_epoch - 1)
            ):
                is_accumulating = True
            else:
                is_accumulating = False

            coords = mrinr.nn.channels_as_coords(
                batch_dict["coords"].to(device), has_batch_dim=True
            )

            # Forward pass
            pred_displacement_field_mm = epinr(
                x=coords,
                grid_fov=subj_data.b0_fov,
                grid_min_coord=subj_data.b0_min_coord,
            )
            pred_deformation_field_mm = coords + pred_displacement_field_mm

            # Sample the fixed t1w image.
            fixed_t1w_sample = sample_t1w(epinr.apply_fixed_rigid_affine(coords))

            # Sample moving image with Jacobian modulation at the predicted deformed
            # coordinates.
            J_pred = jacobian_matrix_batched(
                pred_displacement_net=epinr,
                x_coords=coords,
                grid_fov=subj_data.b0_fov,
                grid_min_coord=subj_data.b0_min_coord,
                jac_of_warp_type="displacement",
                compile_jacobian=True,
            )
            if params.enable_jac_mod_in_training:
                det_j_continuous_displace = 1 + J_pred[:, None, ..., 1, 1]
                moving_b0_sample = det_j_continuous_displace * sample_b0(
                    pred_deformation_field_mm
                )
            else:
                moving_b0_sample = sample_b0(pred_deformation_field_mm)
            b0_scanner_space_weight_mask = torch.maximum(
                sample_b0_weight_mask(coords),
                sample_t1w_weight_mask(epinr.apply_fixed_rigid_affine(coords)),
            )
            # Compute loss.
            loss, loss_term_vals = loss_fn(
                weighted_mse_sim=(
                    moving_b0_sample,
                    fixed_t1w_sample,
                    b0_scanner_space_weight_mask,
                ),
                weighted_mi_sim=(
                    moving_b0_sample,
                    fixed_t1w_sample,
                    b0_scanner_space_weight_mask,
                ),
                weighted_ncc_sim=(moving_b0_sample, fixed_t1w_sample),
                weighted_lncc_sim=(moving_b0_sample, fixed_t1w_sample),
                laplace_ncc_sim=(moving_b0_sample, fixed_t1w_sample),
                laplace_mse_sim=(
                    moving_b0_sample,
                    fixed_t1w_sample,
                    b0_scanner_space_weight_mask,
                ),
                jac_fro_regularization=J_pred,
                bending_energy_regularization=(epinr, coords),
                return_loss_terms=True,
            )

            loss.backward()

            # Record unweighted loss values, along with weighted sum.
            for loss_term, loss_val in (
                loss_term_vals
                | {"weighted_sum": {"value": loss.detach().cpu().item(), "weight": 1.0}}
            ).items():
                accumulate_train_loss[loss_term].append(loss_val)

            if not is_accumulating:
                optim.step()
                optim.zero_grad()
                # Store averages of each loss term for this step.
                for loss_term, acc_loss_vals_weights in accumulate_train_loss.items():
                    l_vals = [l["value"] for l in acc_loss_vals_weights]
                    l_weight = [l["weight"] for l in acc_loss_vals_weights]
                    loss_value = np.mean(l_vals)
                    loss_weight = np.median(l_weight)
                    for l_val, is_weighted in zip(
                        (loss_value, loss_value * loss_weight), (False, True)
                    ):
                        train_loss["epoch"].append(epoch)
                        train_loss["step"].append(step)
                        train_loss["loss_term"].append(loss_term)
                        train_loss["val"].append(l_val)
                        train_loss["weighted"].append(is_weighted)
                        if loss_term == "weighted_sum" and not is_weighted:
                            loss_str = f"{loss_value: 10.9f}"
                            pr(f"{loss_str} | ", end="")
                accumulate_train_loss.clear()
                # Only update step after the model was updated.
                step += 1
            # Always update batch_step, even if accumulating.
            batch_step += 1
        pr()
        pr(f" Epoch {epoch} completed.")
        # Epoch completed, update LR scheduler.
        lr_scheduler.step()
        if epoch == stop_rigid_aff_epoch:
            pr("Stopping fixed rigid affine refinement.")
        if epoch >= stop_rigid_aff_epoch:
            optim.param_groups[1]["lr"] = 0.0
        pr("Learning rate(s):")
        pr([pg["lr"] for pg in optim.param_groups])

        # Validation step
        epinr.eval()
        with torch.no_grad():
            val_results = validation_step(
                epinr=epinr,
                subj_data=subj_data,
                params=params,
                epoch=epoch,
                step=step,
                val_score_fns=val_score_fns,
                b0_spacing=b0_spacing,
                viz_validation_every=viz_validation_every
                if not FULL_EXPERIMENT
                else params.epochs,
                pr=pr,
            )
            val_pred_displacement_fields_list.append(
                val_results["pred_displacement_field_mm"]
            )
            val_pred_undistorted_b0s_list.append(val_results["pred_undistorted_b0"])
            for k, v in val_results["val_scores"].items():
                val_scores["epoch"].append(epoch)
                val_scores["step"].append(step)
                val_scores["metric"].append(k)
                val_scores["val"].append(v)
        pr("\n")

    final_pred_displacement_field_mm = val_pred_displacement_fields_list[-1]
    train_loss = pd.DataFrame.from_dict(train_loss)
    val_scores = pd.DataFrame.from_dict(val_scores)
    pr("Saving training and validation scores...")
    train_loss.to_csv(train_scores_f, index=False)
    val_scores.to_csv(val_scores_f, index=False)
    # Save validation predicted displacement fields and undistorted b0s.
    if not FULL_EXPERIMENT:
        final_val_fig_f.unlink(missing_ok=True)
        val_pred_displacement_fields_f.unlink(missing_ok=True)
        val_undistorted_b0s_f.unlink(missing_ok=True)
    val_pred_displacement_fields_arr = np.stack(
        val_pred_displacement_fields_list, axis=-1
    )
    val_undistorted_b0s_arr = np.stack(val_pred_undistorted_b0s_list, axis=-1)
    pr("Saving validation predicted displacement fields and undistorted b0s...")
    nib.save(
        nib.Nifti1Image(
            val_pred_displacement_fields_arr,
            affine=subj_data.b0_affine.cpu().numpy().squeeze(0),
        ),
        val_pred_displacement_fields_f,
    )
    nib.save(
        nib.Nifti1Image(
            val_undistorted_b0s_arr,
            affine=subj_data.b0_affine.cpu().numpy().squeeze(0),
        ),
        val_undistorted_b0s_f,
    )
    plt.savefig(final_val_fig_f)
    plt.show()

    # Plot training and validation scores.
    if not isinstance(train_loss, pd.DataFrame):
        train_loss = pd.DataFrame.from_dict(train_loss)
        val_scores = pd.DataFrame.from_dict(val_scores)

    sns.relplot(
        train_loss,
        x="step",
        y="val",
        row="weighted",
        col="loss_term",
        hue="loss_term",
        kind="line",
        facet_kws={"sharey": "row", "sharex": True},
    )
    plt.title("Training Losses over Steps")
    plt.savefig(train_loss_plot_f)
    if not FULL_EXPERIMENT:
        plt.show()
    else:
        plt.clf()
    sns.relplot(
        val_scores,
        x="step",
        y="val",
        col="metric",
        hue="metric",
        kind="line",
        facet_kws={"sharey": False, "sharex": True},
    )
    plt.savefig(val_scores_plot_f)
    if not FULL_EXPERIMENT:
        plt.show()
    else:
        plt.clf()

    if FULL_EXPERIMENT:
        # Save out final predicted displacement field.
        final_pred_displacement_mm_f = (
            subj_result_dir
            / f"epinr_final_pred_displace_field_dir-{subj_data.pe_dir}_mm.nii.gz"
        )
        nib.save(
            nib.Nifti1Image(
                final_pred_displacement_field_mm,
                affine=subj_data.b0_affine.cpu().numpy().squeeze(0),
            ),
            final_pred_displacement_mm_f,
        )

        final_pred_suscept_field_hz = (
            (final_pred_displacement_field_mm * (-1 if subj_data.pe_dir == "ap" else 1))
            / b0_spacing[0, 1].cpu().numpy()
        ) / subj_data.total_readout_time_s
        final_pred_displacement_hz_f = (
            subj_result_dir / "epinr_final_pred_suscept_field_hz.nii.gz"
        )
        nib.save(
            nib.Nifti1Image(
                final_pred_suscept_field_hz,
                affine=subj_data.b0_affine.cpu().numpy().squeeze(0),
            ),
            final_pred_displacement_hz_f,
        )
        torch.save(epinr.state_dict(), subj_result_dir / "epinr_final_model.pt")
        pr(f"Finished subject {subj_data.dataset_name}, {subj_data.subj_id}.")

if FULL_EXPERIMENT:
    # Move the tmp result dir to the final result dir.
    print(
        f"Moving temporary result dir\n{tmp_result_dir}\nto final directory\n{final_result_dir}"
    )
    tmp_result_dir.rename(final_result_dir)
print("Done")
# -------------------------------------------------------------------------

### Reconstructed Displacement Field Viz

In [ ]:
import skimage
import skimage.filters

b01 = subj_data.b0.detach().squeeze(0).squeeze(0).cpu().numpy()
b02 = moving_b0_sample.detach().squeeze(0).squeeze(0).cpu().numpy()
b03 = subj_data.topup_corrected_b0.detach().squeeze(0).squeeze(0).cpu().numpy()
t1w1 = fixed_t1w_sample.detach().squeeze(0).squeeze(0).cpu().numpy()

s = 0.5
dog_b0 = skimage.filters.difference_of_gaussians(
    b01,
    low_sigma=s,
)
dog_epinr_b0 = skimage.filters.difference_of_gaussians(
    b02,
    low_sigma=s,
)
# my_dog_epinr_b0 = laplace(moving_b0_sample)
dog_topup_b0 = skimage.filters.difference_of_gaussians(
    b03,
    low_sigma=s,
)
dog_t1w = skimage.filters.difference_of_gaussians(
    # 1 - t1w1,
    t1w1,
    low_sigma=s,
)
# k = 9
# l_b0 = skimage.filters.laplace(b01, ksize=k)
# l_b0 = l_b0 / np.abs(l_b0).sum((-1, -2), keepdims=True)
# l_topup_b0 = skimage.filters.laplace(b03, ksize=k)
# l_topup_b0 = l_topup_b0 / np.abs(l_topup_b0).sum((-1, -2), keepdims=True)
# l_epinr_b0 = skimage.filters.laplace(b02, ksize=k)
# l_t1w = skimage.filters.laplace(t1w1, ksize=k)
# l_t1w = l_t1w / np.abs(l_t1w).sum((-1, -2), keepdims=True)
plt.figure(dpi=150)
mrinr.viz.plot_1_ch_multi_slice_compare_vols(
    dog_b0,
    dog_epinr_b0,
    dog_topup_b0,
    dog_t1w,
    colorbars="rows",
    axial_slice_idx=(0.35, 0.5),
    saggital_slice_idx=0.45,
);